# **Desafio**

## **Descrição do cenário**

A empresa C3 atua no setor financeiro e acompanha o comportamento de ativos negociados em diferentes bolsas de valores ao redor do mundo. Com o aumento da volatilidade dos mercados e o crescimento da quantidade de dados financeiros disponíveis, a empresa busca formas mais inteligentes de orientar suas decisões estratégicas. Visando transformar dados brutos em conhecimento acionável, a C3 reuniu uma base robusta contendo informações históricas de preços, volumes, dividendos, indicadores financeiros e dados setoriais. Apesar da riqueza desses dados, eles ainda não foram analisados de forma estruturada.


---
## **Problema a ser resolvido**

Para potencializar sua atuação, a C3 deseja compreender como diferentes variáveis influenciam o desempenho das ações e quais padrões podem ser extraídos para embasar previsões e escolhas de investimento mais eficazes. Essa iniciativa visa identificar oportunidades ocultas, riscos potenciais e gerar inteligência competitiva baseada em dados. Então, você foi desafiado a analisar esse conjunto de dados e ajudar a C3 a encontrar respostas para as seguintes perguntas:

- Quais são os fatores que mais influenciam o preço de fechamento das ações ao longo do tempo?
- Existe alguma associação entre características setoriais/industriais e o desempenho das ações?
- Como o volume de negociações e os dividendos distribuídos se comportam entre diferentes setores e ao longo do tempo?
- O que pode ser feito para otimizar as vendas?


---
# **ETL (Extract, Transform, Load)**

## **Importação das bibliotecas**

-> Começaremos importando as bibliotecas que vão nos dar as ferramentas necessárias para o processo de ETL:

- Pandas: para manipulação e análise de dados estruturados em tabelas
- NumPy: para realizar operações matemáticas eficientes
- thefuzz: usada para fuzzy matching, ou seja, para comparar strings que são parecidas, mas não exatamente iguais
- re: biblioteca para trabalhar com strings
- matplotlib: usada para criação de gráficos
- google.colab: usada para acessar o google Drive pelo Colab

-> O pip install abaixo é para baixar a biblioteca thefuzz e p drive.mount é para acessar de fato o Drive


In [1]:
!pip install thefuzz[speedup]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.9 MB/s eta 0:00:00


In [2]:
#Importando as libs que serão utilizadas durante o tratamento dos dados
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from google.colab import drive
from thefuzz import fuzz, process

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


---
## **Etapa *Extract***

-> Nessa primeira etapa, coletaremos os dados da base para determinadas variáveis, com as quais trabalharemos no decorrer da ETL

---

### **Base stock_part1**

-> Primeiramente, importaremos a base *stock_part1*, no formato CSV, copiando-a para a variável "base1", que será usada para manipulação e tratamento

In [4]:
# Importando a base de dados stock_part1
base1 = pd.read_csv('/content/drive/MyDrive/pta/stock_part1.csv')

---
-> Agora vamos analisar o tamanho da base1

In [5]:
base1.shape

(200000, 11)

---
-> Agora, faremos uma visualização geral da base olhando apenas as primeiras linhas para entender a estrutura e conteúdo

In [6]:
# Visualizando as 10 primeiras linhas da base
base1.head(10)

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio
0,1,2023-09-03,meta,258.44,909.63,1380.05,"595,86",61.54,"8,846,040,350",3.57,1:1
1,2,"Aug 28, 2024",NVDA,678.96,725.03,185.51,295.83,843.78,7346935699,2.85,3:2
2,3,"May 10, 2022",INTU,1348.94,215.00,831.86,874.05,"$336,19","8,027,589,419",2.28,2:1
3,4,2017-12-26,NVDA,$186.39,1111.85,508.65,511.49,$952.63,1939.5M,4.51,3:2
4,5,"Jan 20, 2022",WORK,1425.94,692.20,340.75,$378.58,1493.44,8984007687,1.93,NaN
5,6,12/12/2016,GOOG,61.76,"262,36",$1315.19,793.41,174.00,1434944091,4.09,3:2
6,7,2015-11-13,tsla,1112.92,"105,69",1342.15,917.22,57.66,4621479471,0.33,NaN
7,8,31/01/2015,NVDA,485.90,190.38,425.99,866.78,1434.97,5081802848,0.33,2:1
8,9,"Feb 14, 2021",NFLX,$676.41,1033.25,"604,71",524.23,350.97,7748945628,2.64,2:1
9,10,2024-11-17,adbe,$522.85,$322.90,373.11,652.81,274.67,8357743507,2.41,2:1


---
-> Assim, podemos definir o **Dicionário dos dados** da base1:

1. **ID**
  - Tipo: numérico
  - Descrição: identificador único de cada linha do dataset, sendo uma sequência

2. **Date**
  - Tipo: string
  - Descrição: data dos dados da ação naquele dia

3. **Ticker**
  - Tipo: string
  - Descrição: código da empresa negociada na bolsa de valores

4. **Open**
  - Tipo: numérico c
  - Descrição: preço da ação no momento de abertura do mercado no dia especificado em "Date"

5. **High**
  - Tipo: numérico (dólar)
  - Descrição: maior valor que a ação atingiu durante o dia especificado em "Date"

6. **Low**
  - Tipo: numérico (dólar)
  - Descrição: menor valor que a ação atingiu no dia especificado em "Date"

7. **Close**
  - Tipo: numérico (dólar)
  - Descrição: preço da ação no fechamento do mercado no dia especificado em "Date"

8. **Adj Close (Adjusted Close)**
  - Tipo: numérico (dólar)
  - Descrição: preço de fechamento ajustado por eventos como dividendos, desdobramentos (splits), e distribuições, sendo mais apropriado para análise de performance ao longo do tempo

9. **Volume**
  - Tipo: numérico
  - Descrição: quantidade de ações negociadas no dia especificado em "Date"

10. **Dividendo**
  - Tipo: numérico (dólar)
  - Descrição: valor do dividendo pago por ação naquele dia

11. **Split Radio**
  - Tipo: string (essência numérica)
  - Descrição: mostra se houve desdobramento (split) ou agrupamento de ações nesse dia ()
  ---




### **Base stock_part2**

-> Primeiramente, importaremos a base *stock_part2*, no formato CSV, copiando-a para a variável "base2", que será usada para manipulação e tratamento

In [7]:
# Importando a base de dados stock_part2 do Drive
base2 = pd.read_csv('/content/drive/MyDrive/pta/stock_part2.csv')

---
-> Agora, faremos uma visualização geral da base olhando apenas as primeiras linhas para entender a estrutura e conteúdo

In [8]:
# Visualizando as 5 primeiras linhas da base
base2.head()

,ID,Sector,Industry,Exchange,Currency,MarketCap,PE Ratio,EPS,Next Earnings,Previous Close,Day Range
0,1,TELECOM,Chemicals,NYSE,USD,"370,815,220,190",27.40,16.68,"May 19, 2025",567.57,X095.58-898.36
1,2,Real Estate,Property Mgmt,TSE,$,1289775810656,90.13,"0,57",06-23-2025,$185.1Z,907.52-552.85
2,3,healthcare,Biotech,LSE,$,97815908000,$15.67,18.17,07-21-2025,823.07,1380.78-1361.69
3,4,Telecom,Electric Power,HKEX,$,1658573090566,83.90,-4.82,23/07/2025,179.88,457.99-138.11
4,5,retail,Banking,NASDAQ,US DOLLAR,315341356874,79.86,5.67,08/06/2025,$1004.79,644.16-920.32


---
-> Assim, podemos definir o **Dicionário dos dados** da base2:

1. **ID**
  - Tipo: numérico
  - Descrição: identificador único de cada linha do dataset, sendo uma sequência

2. **Sector**
  - Tipo: string
  - Descrição: Mostra qual o setor da empresa

3. **Industry**
  - Tipo: string
  - Descrição: Mostra com qual industria a empresa trabalha, sendo uma sub parte de sector

4. **Exchange**
  - Tipo: numérico
  - Descrição: Bolsa de valores onde ações da empresa são negociadas.

5. **Currency**
  - Tipo: numérico
  - Descrição: Qual o tipo de moeda usada na negociação(USD)

6. **Market Cap**
  - Tipo: numérico
  - Descrição: Valor atual da empresa na bolsa de valores

7. **PE Ratio**
  - Tipo: numérico
  - Descrição: Indica quantos dólares os investidores estão pagando por cada dólar de lucro que a empresa gera

8. **EPS**
  - Tipo: numérico (dólar)
  - Descrição: indica quanto de lucro líquido cada ação da empresa gerou no período analisado, dividindo o lucro pela quantidade de ações compradas

9. **Next Earnings**
  - Tipo: data
  - Descrição: significa o próximo anúncio de resultados (lucros) da empresa

10. **Previous Close**
  - Tipo: numérico (dólar)
  - Descrição: é o último preço pelo qual a ação foi negociada no final do dia anterior

11. **Day Range**
  - Tipo: numérico
  - Descrição: mostra a variação de preço da ação ao longo do dia atual de negociação
    ---






---
## **Etapa *Transform***

-> Nessa segunda etapa, faremos a limpeza e padronização dos dados que coletamos

---

### **Base stock_part1**

-> Primeiro vamos verificar se existem valores nulos na nossa base

#### **Análise de dados faltantes**

In [9]:
# Armazena o número de dados faltantes de cada coluna da base na variável "dados_faltantes"
dados_faltantes = base1.isnull().sum()
print(dados_faltantes)

ID                 0
Date               0
Ticker             0
Open               0
High               0
Low                0
Close              0
Adj Close          0
Volume             0
Dividend           0
Split Ratio    38480
dtype: int64


-> Notamos que apenas a coluna Split Ratio apresenta dados faltantes. No subtópico do tratamento específico desta coluna, adotaremos as medidas necessárias

---

#### **Análises de linhas duplicadas**

In [10]:
# Retorna o número de linhas duplicadas
base1.duplicated().sum()

np.int64(0)

-> Notamos que não haverá preocupação em excluir linhas duplicadas

---

#### **Tratamento da coluna Date**

->  Aqui faremos a padronização da coluna Date, deixando as datas com dia, mês e ano, nessa ordem, separados por "-". Demos uma varrida visualmente nos dados e percebemos que as principais diferenças em relação a padronização foram:

- Uso de "/" para separação
- Formato americano
- Mês por escrito
- Número correspondente ao dia/mês irreal (ex: mês 65)

-> Assim, padronizaremos usando um módulo de análises de datas usado para interpretar e converter automaticamente as datas escritas em diferentes formatos para a data padrão "XX-XX-XXXX". Isso será feito em uma variável temporária chamada "base_tmp1" para testar as modificações sem afetar a base original

In [11]:
# importa o módulo de análise de datas chamado parser, que serve para interpretar e converter automaticamente datas escritas em diferentes formatos para data padrão do Python (datetime)
from dateutil import parser

# Cópia da base original para teste
base_tmp1 = base1.copy()

# Função para padronizar datas deixando-as no formato DD-MM-YYYY
def padronizar_data(data):
    try:
        data_convertida = parser.parse(str(data), dayfirst=False)
        return data_convertida.strftime('%d-%m-%Y')
    except Exception:
        return data  # Caso falhe na conversão

# Aplica a função à coluna 'Date'
base_tmp1['Date'] = base_tmp1['Date'].apply(padronizar_data)

---
-> Analisaremos agora o quantitativo de datas ainda não padronizadas (datas com números inexistentes para dia ou mês, ou seja, datas irreais)

In [12]:
# Expressão regular para o formato DD-MM-AAAA
padrao_data = r'^\d{2}-\d{2}-\d{4}$'

# Garante que todos os valores sejam tratados como string
base_tmp1['Date'] = base_tmp1['Date'].astype(str)

# Conta quantas datas NÃO seguem o padrão
datas_nao_padronizadas = ~base_tmp1['Date'].str.match(padrao_data)
quantidade_nao_padronizadas = datas_nao_padronizadas.sum()

print(f"Número de datas não padronizadas: {quantidade_nao_padronizadas}")

Número de datas não padronizadas: 2030


---
-> Agora realizaremos os ajustes para as datas que não estão padronizadas por terem números não correspondentes com dia ou mês, ou seja, por serem irreais.

-> Para isso, substituiremos essas datas errôneas pela moda geral da coluna

-> Primeiramente, vamos encontrar a data modal

In [13]:
# Filtra somente as datas válidas no formato DD-MM-AAAA
datas_validas = base_tmp1['Date'].dropna().astype(str)
datas_formatadas = datas_validas[datas_validas.str.match(padrao_data)]

# Calcula a moda das datas válidas
moda_data = datas_formatadas.mode()

# Exibe o resultado
if not moda_data.empty:
    print(f"Moda da coluna 'Date' (apenas datas válidas): {moda_data.iloc[0]}")
else:
    print("Nenhuma data válida no formato DD-MM-AAAA foi encontrada.")

Moda da coluna 'Date' (apenas datas válidas): 20-06-2017


---
-> Agora faremos a substituição

In [14]:
# Garante que a coluna 'Date' seja string
base_tmp1['Date'] = base_tmp1['Date'].astype(str)

# Identifica as datas válidas
datas_validas = base_tmp1['Date'][base_tmp1['Date'].str.match(padrao_data)]

# Calcula a moda das datas válidas
moda_data = datas_validas.mode().iloc[0] if not datas_validas.empty else None

# Substitui datas inválidas pela moda, se a moda existir
if moda_data:
    # Cria uma máscara para identificar as datas inválidas
    mascara_invalidas = ~base_tmp1['Date'].str.match(padrao_data)

    # Conta quantas substituições serão feitas
    total_substituicoes = mascara_invalidas.sum()

    # Realiza a substituição
    base_tmp1.loc[mascara_invalidas, 'Date'] = moda_data

    print(f"{total_substituicoes} datas inválidas foram substituídas pela moda: {moda_data}")
else:
    print("Nenhuma data válida no formato DD-MM-AAAA foi encontrada para substituição.")

2030 datas inválidas foram substituídas pela moda: 20-06-2017


In [15]:
# Garante que todos os valores sejam tratados como string
base_tmp1['Date'] = base_tmp1['Date'].astype(str)

# Conta quantas datas NÃO seguem o padrão
datas_nao_padronizadas = ~base_tmp1['Date'].str.match(padrao_data)
quantidade_nao_padronizadas = datas_nao_padronizadas.sum()

print(f"Número de datas não padronizadas: {quantidade_nao_padronizadas}")

Número de datas não padronizadas: 0


---

-> Agora vamos converter para o formato da biblioteca datetime () para facilitar nossas futuras análises

In [16]:
# Limpa a coluna Date antes da conversão
base_tmp1['Date'] = base_tmp1['Date'].astype(str).str.strip()

# Converte para datetime com formato explícito
base_tmp1['Date'] = pd.to_datetime(base_tmp1['Date'], format='%d-%m-%Y', errors='coerce')

---
-> Agora vamos visualizar as mudanças na base

In [17]:
base_tmp1.head(10)

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio
0,1,2023-09-03,meta,258.44,909.63,1380.05,"595,86",61.54,"8,846,040,350",3.57,1:1
1,2,2024-08-28,NVDA,678.96,725.03,185.51,295.83,843.78,7346935699,2.85,3:2
2,3,2022-05-10,INTU,1348.94,215.00,831.86,874.05,"$336,19","8,027,589,419",2.28,2:1
3,4,2017-12-26,NVDA,$186.39,1111.85,508.65,511.49,$952.63,1939.5M,4.51,3:2
4,5,2022-01-20,WORK,1425.94,692.20,340.75,$378.58,1493.44,8984007687,1.93,NaN
5,6,2016-12-12,GOOG,61.76,"262,36",$1315.19,793.41,174.00,1434944091,4.09,3:2
6,7,2015-11-13,tsla,1112.92,"105,69",1342.15,917.22,57.66,4621479471,0.33,NaN
7,8,2015-01-31,NVDA,485.90,190.38,425.99,866.78,1434.97,5081802848,0.33,2:1
8,9,2021-02-14,NFLX,$676.41,1033.25,"604,71",524.23,350.97,7748945628,2.64,2:1
9,10,2024-11-17,adbe,$522.85,$322.90,373.11,652.81,274.67,8357743507,2.41,2:1


---
#### **Tratamento das colunas Open, High, Low, Close, Adj Close e Dividend**

-> Aqui faremos a padronização das colunas mencionadas, deixando os números em dólar e na notação numérica com "." separando parte inteira de decimal (convertido para float). Demos uma varrida visualmente nos dados e percebemos que as principais diferenças em relação a padronização foram:
- A presença de vírgula para separar a parte decimal e também a parte de milhar
- A presença do símbolo "$"
- A preseça de letras, como "Z"

-> Assim, esses foram nosso foco na implementação da função de padronização. Porém, sabemos que podem haver outras variações que não notamos. Por isso, resolvemos fazer uma contagem das linhas de cada coluna que foram mantidas por já estarem padronizadas + as linhas de cada coluna que foram padronizadas e convertidas para float e comparar com o número total de linhas de cada coluna. Logo, se não forem iguais é porque houve linhas que não estavam inclusas nas prinicipais diferenças citadas anteriormente e que precisamos tratá-las.

-> Também percebemos que alguns valores da mesma linha em relação a Low e High estão trocados (menor valor em high e maior valor em low) e faremos essas substituição

---
-> Primeiramente, vamos saber a quantidade de linhas de cada coluna

In [18]:
# Armazenando os nomes de cada coluna na variável "colunas"
colunas = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Dividend']

print("Quantidade de linhas de cada coluna: \n")

# Itera sobre cada coluna printando o nome + número de linhas dela
for col in colunas:
  print(f"{col}: {len(base1[colunas])}")


Quantidade de linhas de cada coluna: 

Open: 200000
High: 200000
Low: 200000
Close: 200000
Adj Close: 200000
Dividend: 200000


---
-> Agora, implementaremos a função de padronização e a contagem de linhas já padronizadas + as que foram padronizadas em uma variável temporária chamada "base_tmp" para testar as modificações sem afetar a base original


In [19]:
# Biblioteca para trabalhar com strings
import re

# Função de padronização com contador
def padronizar_valor(valor):
    valor_original = valor

    valor_str = str(valor).strip()

    # Mantém o valor original se tiver letras/símbolos não permitidos (exceto $)
    if re.search(r"[^\d.,$-]", valor_str):
        return valor_original  # não altera

    # Remove cifrão
    valor_str = valor_str.replace('$', '')

    # Converte vírgula decimal para ponto se for o caso
    if re.match(r"^\d+,\d{2}$", valor_str):
        valor_str = valor_str.replace(',', '.')

    # Remove separadores de milhar
    valor_str = valor_str.replace(',', '')

    try:
        return float(valor_str)
    except ValueError:
        return valor_original  # retorna o original se não puder converter


print("Quantidade de linhas padronizadas de cada coluna: \n")

for col in colunas:
    # Aplica padronização
    base_tmp1[col] = base_tmp1[col].apply(padronizar_valor)

    # Conta quantos valores são float (incluindo os que já estavam como float)
    padronizados_col = base_tmp1[col].apply(lambda x: isinstance(x, float)).sum()
    print(f"{col}: {padronizados_col} valores padronizados")

Quantidade de linhas padronizadas de cada coluna: 

Open: 195763 valores padronizados
High: 195691 valores padronizados
Low: 195696 valores padronizados
Close: 195666 valores padronizados
Adj Close: 195744 valores padronizados
Dividend: 196597 valores padronizados


---
-> Agora, vamos calcular os valores que não são float (não padronizados) de cada coluna

In [20]:
# Dicionário para armazenar contagem de não-floats por coluna
nao_float_count = {}

# Loop pelas colunas para verificar tipos
for col in colunas:
    count = base_tmp1[col].apply(lambda x: not isinstance(x, float)).sum()
    nao_float_count[col] = count

# Exibir resumo por coluna
print("Quantidade de valores que NÃO são float em cada coluna:")
for col, count in nao_float_count.items():
    print(f"{col}: {count}")

# Total geral
total_nao_float = sum(nao_float_count.values())
print(f"\nTOTAL de valores não padronizados como float: {total_nao_float}")

Quantidade de valores que NÃO são float em cada coluna:
Open: 4237
High: 4309
Low: 4304
Close: 4334
Adj Close: 4256
Dividend: 3403

TOTAL de valores não padronizados como float: 24843


---
-> Agora vamos confirmar que esses valores são justamente o que contém 'Z'

In [21]:
def contar_valores_com_Z(df):
    colunas_alvo = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Dividend']
    contagem_total = 0

    for coluna in colunas_alvo:
        # Converte para string, verifica se contém 'Z' e soma os casos verdadeiros
        contagem_coluna = df[coluna].astype(str).str.contains('Z', case=False, na=False).sum()
        print(f"{coluna}: {contagem_coluna} valores com 'Z'")
        contagem_total += contagem_coluna

    print(f"Total de valores com 'Z': {contagem_total}")
    return contagem_total

contar_valores_com_Z(base_tmp1)

Open: 4237 valores com 'Z'
High: 4309 valores com 'Z'
Low: 4304 valores com 'Z'
Close: 4334 valores com 'Z'
Adj Close: 4256 valores com 'Z'
Dividend: 3403 valores com 'Z'
Total de valores com 'Z': 24843


np.int64(24843)

---
-> Agora restringimos nosso tratamento apenas para os valores com 'Z', garantindo que não há outra variação
-> Optamos por análises discutidas que substituiremos o 'Z' por '2', pois concluímos que é o que mais faz sentido e também porque se for comparar com a substituição por '0', essa última seria mais significativa, podendo afetar de forma mais significativa nossas análises futuras

In [22]:
def substituir_Z_por_2(df):
    colunas_alvo = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Dividend']
    substituicoes = 0

    for coluna in colunas_alvo:
        # Garante que os dados sejam tratados como strings
        df[coluna] = df[coluna].astype(str)

        # Conta quantos valores contêm 'Z' antes da substituição
        com_Z = df[coluna].str.contains('Z', case=False, na=False)
        substituicoes_coluna = com_Z.sum()

        # Substitui 'Z' por '2'
        df.loc[com_Z, coluna] = df.loc[com_Z, coluna].str.replace('Z', '2', case=False, regex=False)

        print(f"{coluna}: {substituicoes_coluna} substituições feitas")
        substituicoes += substituicoes_coluna

    print(f"Total de substituições realizadas: {substituicoes}")
    return df

base_tmp1 = substituir_Z_por_2(base_tmp1)

Open: 4237 substituições feitas
High: 4309 substituições feitas
Low: 4304 substituições feitas
Close: 4334 substituições feitas
Adj Close: 4256 substituições feitas
Dividend: 3403 substituições feitas
Total de substituições realizadas: 24843


---
-> Agora, vamos verificar se ainda resta algum símbolo que possa atrapalhar a conversão para o tipo numérico:

In [23]:
def encontrar_simbolos(df, coluna):
    if coluna not in df.columns:
        print(f"Coluna '{coluna}' não encontrada.")
        return set()

    simbolos = set()
    valores = df[coluna].dropna().astype(str)

    for valor in valores:
        encontrados = re.findall(r'[^0-9\s]', valor)  # tudo que não for número nem espaço
        simbolos.update(encontrados)

    return simbolos

for col in colunas:
      print(encontrar_simbolos(base_tmp1, col))


{',', '.', '$'}
{',', '.', '$'}
{',', '.', '$'}
{',', '.', '$'}
{',', '.', '$'}
{',', '.', '$'}


---
-> Os símbolos '$' e ',' podem atrapalhar nossa conversão numérica, vamos corrigir isso:

In [24]:
for col in colunas:
    # Aplica padronização
    base_tmp1[col] = base_tmp1[col].apply(padronizar_valor)

    # Conta quantos valores são float (incluindo os que já estavam como float)
    padronizados_col = base_tmp1[col].apply(lambda x: isinstance(x, float)).sum()
    print(f"{col}: {padronizados_col} valores padronizados")

Open: 200000 valores padronizados
High: 200000 valores padronizados
Low: 200000 valores padronizados
Close: 200000 valores padronizados
Adj Close: 200000 valores padronizados
Dividend: 200000 valores padronizados


---

-> Agora, vamos fazer as alterações entre valores trocados da coluna High e Low

In [25]:
# converte pra float
base_tmp1['Low'] = pd.to_numeric(base_tmp1['Low'], errors='coerce')
base_tmp1['High'] = pd.to_numeric(base_tmp1['High'], errors='coerce')

# contagem de casos onde Low > High
numCasos = base_tmp1['Low'] > base_tmp1['High']

print(f"Linhas com Low > High encontradas: {numCasos.sum()}")

# troca os valores
temp = base_tmp1.loc[numCasos, 'Low']
base_tmp1.loc[numCasos, 'Low'] = base_tmp1.loc[numCasos, 'High']
base_tmp1.loc[numCasos, 'High'] = temp


Linhas com Low > High encontradas: 99777


-> Além disso, encontramos outros tipos de inconsistência nessa base de dados, como por exemplo:

Valor de Open/Close > High ou < Low, vamos tratar esses casos

In [26]:
#Podemos ver que há 4 casos, vamos olhar a quantidade de cada um:

close_high = base_tmp1['Close'] > base_tmp1['High']
close_low = base_tmp1['Close'] < base_tmp1['Low']
open_high = base_tmp1['Open'] > base_tmp1['High']
open_low = base_tmp1['Open'] < base_tmp1['Low']

print(f"Linhas com Close > High encontradas: {close_high.sum()}")
print(f"Linhas com Close < Low encontradas: {close_low.sum()}")
print(f"Linhas com Open > High encontradas: {open_high.sum()}")
print(f"Linhas com Open < Low encontradas: {open_low.sum()}")

Linhas com Close > High encontradas: 66603
Linhas com Close < Low encontradas: 66496
Linhas com Open > High encontradas: 67240
Linhas com Open < Low encontradas: 66380


Conseguimos notar a alta incidência desses casos em nosso dataset, vamos corrigir caso a caso

In [27]:
# Corrigir Open > High
mask_open_gt_high = base_tmp1['Open'] > base_tmp1['High']
base_tmp1.loc[mask_open_gt_high, ['Open', 'High']] = base_tmp1.loc[mask_open_gt_high, ['High', 'Open']].values

# Corrigir Open < Low
mask_open_lt_low = base_tmp1['Open'] < base_tmp1['Low']
base_tmp1.loc[mask_open_lt_low, ['Open', 'Low']] = base_tmp1.loc[mask_open_lt_low, ['Low', 'Open']].values

# Corrigir Close > High
mask_close_gt_high = base_tmp1['Close'] > base_tmp1['High']
base_tmp1.loc[mask_close_gt_high, ['Close', 'High']] = base_tmp1.loc[mask_close_gt_high, ['High', 'Close']].values

# Corrigir Close < Low
mask_close_lt_low = base_tmp1['Close'] < base_tmp1['Low']
base_tmp1.loc[mask_close_lt_low, ['Close', 'Low']] = base_tmp1.loc[mask_close_lt_low, ['Low', 'Close']].values


Agora, vamos verificar se ainda há alguma inconsistência

In [28]:
close_high = base_tmp1['Close'] > base_tmp1['High']
close_low = base_tmp1['Close'] < base_tmp1['Low']
open_high = base_tmp1['Open'] > base_tmp1['High']
open_low = base_tmp1['Open'] < base_tmp1['Low']
numCasos = base_tmp1['Low'] > base_tmp1['High']

print(f"Linhas com Close > High encontradas: {close_high.sum()}")
print(f"Linhas com Close < Low encontradas: {close_low.sum()}")
print(f"Linhas com Open > High encontradas: {open_high.sum()}")
print(f"Linhas com Open < Low encontradas: {open_low.sum()}")
print(f"Linhas com Low > High encontradas: {numCasos.sum()}")





Linhas com Close > High encontradas: 0
Linhas com Close < Low encontradas: 0
Linhas com Open > High encontradas: 0
Linhas com Open < Low encontradas: 0
Linhas com Low > High encontradas: 0


-> Agora vamos confirmar que está tudo correto como float

In [29]:
for col in colunas:
      print(encontrar_simbolos(base_tmp1, col))

{'.'}
{'.'}
{'.'}
{'.'}
{'.'}
{'.'}


-> Por fim, converteremos para numérico todas essas colunas

In [30]:
for col in colunas:
    base_tmp1[col] = pd.to_numeric(base_tmp1[col], errors='coerce')

---
-> Agora, faremos uma visualização geral das mudanças

In [31]:
# Printa as 14 primeiras linhas da base modificada
base_tmp1.head(14)

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio
0,1,2023-09-03,meta,909.63,1380.05,258.44,595.86,61.54,"8,846,040,350",3.57,1:1
1,2,2024-08-28,NVDA,678.96,725.03,185.51,295.83,843.78,7346935699,2.85,3:2
2,3,2022-05-10,INTU,831.86,1348.94,215.00,874.05,336.19,"8,027,589,419",2.28,2:1
3,4,2017-12-26,NVDA,508.65,1111.85,186.39,511.49,952.63,1939.5M,4.51,3:2
4,5,2022-01-20,WORK,692.20,1425.94,340.75,378.58,1493.44,8984007687,1.93,NaN
5,6,2016-12-12,GOOG,262.36,1315.19,61.76,793.41,174.00,1434944091,4.09,3:2
6,7,2015-11-13,tsla,1112.92,1342.15,105.69,917.22,57.66,4621479471,0.33,NaN
7,8,2015-01-31,NVDA,425.99,866.78,190.38,485.90,1434.97,5081802848,0.33,2:1
8,9,2021-02-14,NFLX,676.41,1033.25,524.23,604.71,350.97,7748945628,2.64,2:1
9,10,2024-11-17,adbe,373.11,652.81,322.90,522.85,274.67,8357743507,2.41,2:1


---
#### **Tratamento da Coluna Ticker**


-> Nessa coluna, devemos corrigir e padronizar os nomes das empresas. Ao varrer um pouco o dataset, encontramos as seguintes questões:

* Sigla da empresa com letras minúsculas
* Erros na digitação das siglas das empresas - 1 letra errada...

-> Para corrigirmos esse problema, iniciamos tornando todas as letras minúsculas, reduzindo ao menos um pouco as variações para iniciar.

-> Para que conseguissemos distinguir quais siglas são de uma empresa ou não, listamos as ocorrências de cada sigla nesse dataset.

In [32]:
#Normalizando as siglas - Retirando espaços excessivos e tornando tudo maiúsculo
base_tmp1['Ticker'] = base_tmp1['Ticker'].str.upper().str.strip()

variacoes_siglas = base_tmp1['Ticker'].value_counts().reset_index()
variacoes_siglas.columns = ['variacao', 'frequencia']

#Para que possamos ver todas as siglas e sua frequência
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print(variacoes_siglas)

     variacao  frequencia
0          ZM        6747
1          SQ        6694
2        PYPL        6477
3        SHOP        6450
4        META        6449
5        BABA        6440
6        SNAP        6423
7        WORK        6419
8         AMD        6384
9        CSCO        6372
10       ORCL        6367
11       AAPL        6364
12       INTC        6359
13       TSLA        6354
14       NVDA        6351
15       LYFT        6347
16        IBM        6346
17       SPOT        6346
18        VMW        6336
19       ADBE        6318
20       TWTR        6303
21       PINS        6299
22       MSFT        6298
23       INTU        6286
24       GOOG        6285
25       DOCU        6280
26       NFLX        6276
27        CRM        6266
28       AMZN        6251
29       UBER        6109
30       INTF          14
31        EMD          11
32       INTV          10
33        RBM          10
34       BYPL          10
35       SAOT          10
36       BMZN          10
37       INT

-> Podemos notar que 30 siglas foram citadas mais de 6000 vezes nesse dataset, enquanto todas as outras foram citadas 14 ou menos vezes.

-> A partir disso, podemos concluir que são 30 empresas que foram listadas nesse dataset e as demais são variações erradas, uma vez que a diferença é muito grande (6000 para 14).

-> A partir disso, vamos agrupar as siglas por grau de semelhança com objetivo de fazer a correção das mais siglas e padronizar nosso dataset.


In [33]:
#Utilizando a função criada por Jairo com adição do retorno agora, uma vez que manipularemos os grupos
def Verifica_Categorias_base_retorno(dataframe, categoria):

    if categoria not in dataframe.columns:
        print(f"Erro: a categoria '{categoria}' não foi encontrada no DataFrame.")
        return []

    valores = dataframe[categoria].dropna().astype(str).str.strip().unique().tolist()  # Aqui estou pegando todos os nomes da lista Sector
    grupos = []

    while valores:
        base = valores.pop(0)
        grupo = [base]
        similares = []

        for v in valores:
            if fuzz.ratio(base, v) >= 60:  # Pega apenas os valores mais próximos para comparação
                grupo.append(v)
                similares.append(v)

        # Remove os que já foram agrupados
        valores = [v for v in valores if v not in similares]
        grupos.append(grupo)

    # Exibe os grupos
    for i, grupo in enumerate(grupos, 1):
        print(f"Grupo {i}: {grupo}")

    return grupos  # <-- retorno adicionado aqui


grupos_siglas = Verifica_Categorias_base_retorno(variacoes_siglas, 'variacao')


Grupo 1: ['ZM', 'ZMW', 'ZRM', 'AMZM', 'IZM', 'ZMD', 'CZM', 'ZMZN', 'ZBM']
Grupo 2: ['SQ', 'SQOP', 'SNQP', 'SPQT', 'SQAP', 'SQOT', 'TSLQ', 'TSQA', 'MSQT', 'SPOQ', 'CSQO', 'CSCQ', 'SHOQ', 'SHQP', 'SNAQ']
Grupo 3: ['PYPL', 'BYPL', 'XYPL', 'PBPL', 'PXPL', 'PYPM', 'PYPS', 'WYPL', 'PTPL', 'PAPL', 'PLPL', 'PYDL', 'HYPL', 'GYPL', 'PRPL', 'PYPT', 'PYPQ', 'PYPW', 'PYPB', 'PUPL', 'PQPL', 'PYYL', 'PYPF', 'PYTL', 'ZYPL', 'PYEL', 'RYPL', 'YYPL', 'PMPL', 'PNPL', 'DYPL', 'APPL', 'PYIL', 'YAPL', 'PHPL', 'PYPX', 'PGPL', 'PEPL', 'PPPL', 'PYUL', 'PIPL', 'PWPL', 'UYPL', 'PYPY', 'PYPN', 'PYVL', 'PYCL', 'POPL', 'PDPL', 'QYPL', 'PYAL', 'TYPL', 'PCPL', 'PYHL', 'PYPZ', 'PYPC', 'PYKL', 'PZPL', 'PYPV', 'KYPL', 'PYPO', 'PYLL', 'PJPL', 'PYPU', 'PVPL', 'PYWL', 'IYPL', 'PYOL', 'EYPL', 'PYNL', 'MYPL', 'PYPH', 'PYJL', 'PYRL', 'PYPG', 'PFPL', 'LYPL', 'JYPL', 'PKPL', 'PYPA', 'VYPL', 'FYPL', 'PYFL', 'NYPL', 'PYPR', 'PYPE', 'PYBL', 'PYPI', 'AYPL', 'PYXL', 'SYPL', 'PSPL', 'PYSL', 'PYQL', 'PYPK', 'CYPL', 'OYPL', 'PYZL', 'PYP

-> Agora, podemos notar que esse algoritmo nos retornou 32 grupos de palavra, entretanto, sabemos que são 30 empresas. Apesar do algoritmo auxiliar bastante nesse processo, ele ainda comete algumas falhas, devido ao alto grau de semelhança entre algumas palavras.

-> A partir disso, é necessário que realizemos uma espécie de inspeção manual, para corrigir algumas poucas falhas desse algoritmo.

-> Esse processo ocorrerá analisando cada linha desse grupo de siglas e realocando manualmente para o outro grupo, além de excluir do grupo errado

In [34]:
#Removendo os nomes dos grupos errados e atribuindo nos grupos corretos

grupos_siglas[18].extend(['BMW', 'IMW', 'ZMW'])
grupos_siglas[26].extend(['ZRM','CZM', 'ARM', 'CAM', 'BRM', 'CVM'])
grupos_siglas[27].extend(['AMZM', 'ZMZN', 'AMDN', 'AMZD'])
grupos_siglas[16].extend(['IZM','ZBM', 'ABM', 'IAM'])
grupos_siglas[8].append('ZMD')

palavras_removidas = ['ZMW', 'ZRM', 'CZM', 'AMZM', 'ZMZN', 'IZM', 'ZBM','ZMD']
grupos_siglas[0] = [s for s in grupos_siglas[0] if s not in palavras_removidas]

grupos_siglas[6].extend(['SNQP', 'SQAP','SNAQ'])
grupos_siglas[17].extend(['SPQT', 'SQOT', 'SPOQ', 'SCOT'])
grupos_siglas[13].extend(['TSLQ', 'TSQA', 'TSNA'])
grupos_siglas[9].extend(['CSQO', 'CSCQ', 'CSHO'])
grupos_siglas[3].extend(['SHOQ', 'SHQP', 'SQOP'])
grupos_siglas[22].extend(['MSQT', 'MSOT', 'MSPT'])

palavras_removidas = ['SQOP', 'SNQP', 'SPQT', 'SQAP', 'SQOT', 'TSLQ', 'TSQA', 'MSQT', 'SPOQ', 'CSQO', 'CSCQ', 'SHOQ', 'SHQP', 'SNAQ']
grupos_siglas[1] = [s for s in grupos_siglas[1] if s not in palavras_removidas]

grupos_siglas[11].extend(['APPL', 'NAPL'])
grupos_siglas[2].remove('APPL')
grupos_siglas[3].remove('CSHO')
grupos_siglas[6].remove('TSNA')
grupos_siglas[6].remove('NAPL')

grupos_siglas[20].append('TWOR')
grupos_siglas[10].extend(['ORKL','ORCK'])

palavras_removidas = ['TWOR','ORKL','ORCK']

grupos_siglas[7] = [s for s in grupos_siglas[7] if s not in palavras_removidas]

palavras_removidas = ['ABM', 'IAM', 'ARM', 'CAM', 'AMDN', 'AMZD']

grupos_siglas[8] = [s for s in grupos_siglas[8] if s not in palavras_removidas]

grupos_siglas[9].remove('SCOT')

grupos_siglas[24].append('DOCL')
grupos_siglas[10].remove('DOCL')

grupos_siglas[29].extend(['INTU','INSU','INDU', 'INQU', 'INHU', 'INRU', 'INIU', 'INAU', 'INFU', 'INUU', 'INWU', 'INNU', 'INOU', 'INZU', 'INYU', 'INBU', 'INJU', 'INVU', 'INEU', 'INLU', 'INMU', 'INGU', 'INPU', 'INKU', 'MNTU', 'JNTU', 'HNTU', 'LNTU', 'FNTU', 'BNTU', 'PNTU', 'ANTU', 'TNTU', 'ENTU', 'KNTU', 'GNTU', 'DNTU', 'QNTU', 'RNTU', 'WNTU', 'ZNTU', 'YNTU', 'ONTU', 'UNTU', 'NNTU', 'VNTU', 'CNTU'])
grupos_siglas[12].remove('INTU')

grupos_siglas[22].append('MLFT')
grupos_siglas[15].remove('MLFT')

palavras_removidas = ['BMW','IMW','BRM']
grupos_siglas[16] = [s for s in grupos_siglas[16] if s not in palavras_removidas]

grupos_siglas[17].remove('MSOT')
grupos_siglas[17].remove('MSPT')
grupos_siglas[18].remove('CVM')

grupos_siglas[28].extend(['DBER','ABER'])
grupos_siglas[19].remove('DBER')
grupos_siglas[19].remove('ABER')
grupos_siglas[29].append('INSU')
grupos_siglas[21].remove('INSU')

del grupos_siglas[31]
del grupos_siglas[30]

-> Uma vez realizada a seleção manual, vamos rever como está nossos grupos de siglas no momento

In [35]:
for i, grupo in enumerate(grupos_siglas, 1):
        print(f"Grupo {i}: {grupo}")

Grupo 1: ['ZM']
Grupo 2: ['SQ']
Grupo 3: ['PYPL', 'BYPL', 'XYPL', 'PBPL', 'PXPL', 'PYPM', 'PYPS', 'WYPL', 'PTPL', 'PAPL', 'PLPL', 'PYDL', 'HYPL', 'GYPL', 'PRPL', 'PYPT', 'PYPQ', 'PYPW', 'PYPB', 'PUPL', 'PQPL', 'PYYL', 'PYPF', 'PYTL', 'ZYPL', 'PYEL', 'RYPL', 'YYPL', 'PMPL', 'PNPL', 'DYPL', 'PYIL', 'YAPL', 'PHPL', 'PYPX', 'PGPL', 'PEPL', 'PPPL', 'PYUL', 'PIPL', 'PWPL', 'UYPL', 'PYPY', 'PYPN', 'PYVL', 'PYCL', 'POPL', 'PDPL', 'QYPL', 'PYAL', 'TYPL', 'PCPL', 'PYHL', 'PYPZ', 'PYPC', 'PYKL', 'PZPL', 'PYPV', 'KYPL', 'PYPO', 'PYLL', 'PJPL', 'PYPU', 'PVPL', 'PYWL', 'IYPL', 'PYOL', 'EYPL', 'PYNL', 'MYPL', 'PYPH', 'PYJL', 'PYRL', 'PYPG', 'PFPL', 'LYPL', 'JYPL', 'PKPL', 'PYPA', 'VYPL', 'FYPL', 'PYFL', 'NYPL', 'PYPR', 'PYPE', 'PYBL', 'PYPI', 'AYPL', 'PYXL', 'SYPL', 'PSPL', 'PYSL', 'PYQL', 'PYPK', 'CYPL', 'OYPL', 'PYZL', 'PYPJ']
Grupo 4: ['SHOP', 'SHHP', 'SHOT', 'SHLP', 'SHUP', 'SHOB', 'SHOW', 'DHOP', 'SHOK', 'SNOP', 'IHOP', 'SBOP', 'YHOP', 'SKOP', 'SHXP', 'SHOX', 'LHOP', 'SPOP', 'JHOP', 'SHOO', 'AHO

-> Agora que temos os grupos separados, iremos pegar os nomes corretos das empresas, o que servirá para realizarmos a correção de modo automático.

In [36]:
#Nessa etapa do código, estamos colocando em um vetor as siglas corretas, as quais serão utéis para substituição das siglas erradas mais a frente
nomes_empresas = variacoes_siglas['variacao']
nomes_corretos_empresas = []

for i in range(30):
    nome = nomes_empresas[i]

    nomes_corretos_empresas.append(nome)

nomes_corretos_empresas.remove('INTU')
nomes_corretos_empresas.append('INTU')
print(nomes_corretos_empresas)

['ZM', 'SQ', 'PYPL', 'SHOP', 'META', 'BABA', 'SNAP', 'WORK', 'AMD', 'CSCO', 'ORCL', 'AAPL', 'INTC', 'TSLA', 'NVDA', 'LYFT', 'IBM', 'SPOT', 'VMW', 'ADBE', 'TWTR', 'PINS', 'MSFT', 'GOOG', 'DOCU', 'NFLX', 'CRM', 'AMZN', 'UBER', 'INTU']


-> Agora, basta rodarmos o algoritmo abaixo e todas as siglas serão corrigidas

In [37]:
def padronizar_siglas_por_grupo(df, coluna, nomes_corretos, grupos):

    mapa = {}
    for sigla_correta, grupo in zip(nomes_corretos, grupos):
        for variacao in grupo:
            mapa[variacao] = sigla_correta

    df[coluna] = df[coluna].map(mapa).fillna(df[coluna])
    return df

In [38]:
base1_tempp = padronizar_siglas_por_grupo(base_tmp1, 'Ticker', nomes_corretos_empresas, grupos_siglas)

variacoes_siglas = base_tmp1['Ticker'].value_counts().reset_index()
variacoes_siglas.columns = ['variacao', 'frequencia']
print(variacoes_siglas)

   variacao  frequencia
0      PYPL        6805
1      SHOP        6784
2      BABA        6775
3      META        6767
4        ZM        6747
5      INTC        6742
6      SNAP        6731
7      WORK        6728
8       AMD        6718
9      CSCO        6715
10       SQ        6694
11     AAPL        6693
12      IBM        6688
13     ORCL        6681
14     TSLA        6679
15     NVDA        6667
16     SPOT        6666
17     LYFT        6661
18     ADBE        6643
19     PINS        6635
20     TWTR        6635
21      VMW        6624
22     GOOG        6605
23     NFLX        6605
24     MSFT        6599
25      CRM        6593
26     DOCU        6592
27     AMZN        6564
28     INTU        6517
29     UBER        6447


-> Podemos notar que agora em nosso dataset só há 30 siglas, o que corresponde exatamente a nosso quantitativo de empresas, facilitando extrair insights de cada empresa.



---
#### **Tratamento da coluna Volume**

-> Aqui faremos a padronização da coluna volume. Ao analisarmos, notamos as seguintes distinções:

 * Números terminando com letras/palavras (M, B, K, b, Billion)
 * Números com "," como separador decimal


 -> Logo, padronizaremos para que fiquem apenas os valores e convertê-los, de modo a facilitar a extração de insights mais na frente.

In [39]:

base_tmp1['Volume'] = base_tmp1['Volume'].str.replace(',','') #Tirando a vírgula como separador decimal para facilitar a conversão para número mais a frente
base_tmp1['Volume'].head()


,Volume
0,8846040350
1,7346935699
2,8027589419
3,1939.5M
4,8984007687


-> Logo de cara, conseguimos notar que há letras acompanhando os números, o que nos sugere uma possível multiplicação para que possamos padronizar essa coluna. Com isso em mente, vamos analisar todos os sufixos presentes na tabela

In [40]:
def extrair_sufixos_unicos(coluna):
    sufixos = set()

    for valor in coluna.dropna():
        # Extrai a parte não numérica (sufixo) após os números e pontos
        match = re.findall(r'(?:\d+\.?\d*\s*)([a-zA-Z]+)', str(valor))
        sufixos.update([m.strip() for m in match])

    return sorted(sufixos)

sufixos_volume = extrair_sufixos_unicos(base_tmp1['Volume'])
print(sufixos_volume)

['Billion', 'K', 'M', 'MK', 'Mb', 'b']


-> Podemos notar a quantidade de sufixos distintos, entretanto, ao dar uma analisada visual na tabela, notamos que nem sempre essa multiplicação deve ocorrer, uma vez que algumas multiplicações acabaria nos retornando resultados que destoam do normal dessa coluna. Vejamos:

In [41]:
def contar_tamanhos_inteiros_com_sufixo(coluna):

    resultados = set()

    for valor in coluna.dropna():
        # Ex: '123M', '4.5B', '100 K', '8 billion'
        matches = re.findall(r'(\d+)(?:\.\d+)?\s*([a-zA-Z]+)', str(valor))
        for numero, sufixo in matches:
            resultados.add((len(numero), sufixo))

    return resultados

contar_tamanhos_inteiros_com_sufixo(base_tmp1['Volume'])

{(1, 'M'),
 (2, 'M'),
 (2, 'MK'),
 (2, 'Mb'),
 (3, 'M'),
 (3, 'MK'),
 (3, 'Mb'),
 (4, 'M'),
 (4, 'MK'),
 (4, 'Mb'),
 (7, 'Billion'),
 (7, 'b'),
 (8, 'Billion'),
 (8, 'K'),
 (8, 'b'),
 (9, 'Billion'),
 (9, 'K'),
 (9, 'b'),
 (10, 'Billion'),
 (10, 'K'),
 (10, 'b')}

-> Essa função nos retorna as variações que encontramos na coluna de números inteiro e os sufixos que as acompanham.

-> Podemos notar que algumas multiplicações iriam extrapolar o número de dígitos médios (que é 10), então faremos uma verificação do sufixo e o resultado da multiplicação de acordo com o sufixo, se não extrapolar o número médio de dígitos.

-> Em outros casos, basta a remoção do sufixo para ficar no padrão desejado, além da conversão de float. Utilizaremos a função abaixo, que fará isso tudo:


In [42]:
def converter_volume(base, coluna):
    multiplicadores = {
        'K': 1_000,
        'M': 1000_000,
        'B': 1000_000_000,
        'b': 1000_000_000,
        'billion': 1000_000_000,
        'MK': 1000_000,
        'Mb': 1000_000
    }

    def aplica(valor):
        match = re.search(r'(\d+(?:\.\d+)?)\s*([a-zA-Z]+)', str(valor))
        if match:
            numero = float(match.group(1))
            sufixo = match.group(2)
            fator = multiplicadores.get(sufixo, 1)
            res_mult = numero * fator
            if numero >=1e7 or res_mult > 1e10:
                return int(numero)
            else:
                return int(res_mult)
        else:
            return int(valor)

    base[coluna] = base[coluna].apply(aplica)
    return base


In [43]:
base_tmp1 = converter_volume(base_tmp1, 'Volume') #Realizando a correção da coluna volume

-> Agora, iremos realizar o teste para ver se os sufixos sumiram

In [44]:
teste = extrair_sufixos_unicos(base_tmp1['Volume'])
print(teste)

[]


-> Não há mais sufixos nessa coluna, agora iremos realizar a contagem de quantos números de quantos dígitos existem nessa coluna

In [45]:
from collections import Counter

def contar_inteiros_por_digitos(lista):

    contador = Counter()

    for item in lista:
        if isinstance(item, int):
            num_digitos = len(str(item))
            contador[num_digitos] += 1

    return dict(contador)

contador = contar_inteiros_por_digitos(base_tmp1['Volume'])
print(contador)

{10: 179972, 9: 18050, 7: 196, 8: 1782}


-> Podemos ver que agora só há números com 7 ou mais dígitos, o que corresponde a realidade de um volume, tornando nossa tabela coerente.



---
#### **Tratamento da coluna Split Ratio**

->  Aqui faremos a padronização da coluna Split Ratio, deixando-a padronizada no formato "x:y", onde x e y são números inteiros. Demos uma varrida visualmente nos dados e percebemos que as principais diferenças em relação a padronização foram:

- Uso de "/" e outros caracteres para separação
- Letras no lugar de números
- Muitos dados nulos

-> Assim, discutimos que a padronização de dados nulos e com letras no lugar de números será feita substituindo-os pelo valor modal de split ratio do respectivo Ticker de cada um

-> Primeiramente, padronizaremos apenas os valores com caracter separando os números

In [46]:
# função para a padronização
def padronizar_split_ratio(valor):
    valor_str = str(valor).strip()

    # verifica se há dois números separados por caractere
    if re.match(r'^\d+[^\d]\d+$', valor_str):
        novo_valor = re.sub(r'(?<=\d)[^\d](?=\d)', ':', valor_str)
        # conta como alteração só se o valor mudou
        if novo_valor != valor_str:
            return novo_valor, True
    return valor_str, False


# inicializa contador
contador_alteracoes = 0

# lista que armazena os novos valores
novos_valores = []

for valor in base_tmp1['Split Ratio']:
    novo_valor, alterado = padronizar_split_ratio(valor)
    novos_valores.append(novo_valor)
    if alterado:
        contador_alteracoes += 1

# atualização da coluna
base_tmp1['Split Ratio'] = novos_valores

print(f'Número de valores corrigidos na coluna "Split Ratio": {contador_alteracoes}')

Número de valores corrigidos na coluna "Split Ratio": 2592


---

-> Agora, substituiremos os valores nulos e com letras no lugar de números pelo valor modal de split ratio do respectivo Ticker de cada um

In [47]:
# função que verifica se o valor está no padrão "int:int"
def split_ratio_valido(valor):
    if pd.isnull(valor):
        return False
    return bool(re.match(r'^\d+:\d+$', str(valor).strip()))

# função que calcula moda válida do split ratio por Ticker
moda_por_ticker = (
    base_tmp1[base_tmp1['Split Ratio'].apply(split_ratio_valido)]
    .groupby('Ticker')['Split Ratio']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
)

# função que substitui valores inválidos ou nulos pela moda do ticker
def substituir_split_ratio(row):
    valor = row['Split Ratio']
    if not split_ratio_valido(valor):
        return moda_por_ticker.get(row['Ticker'], valor)
    return valor

# contagem das substituições
split_ratio_antes = base_tmp1['Split Ratio'].copy()
base_tmp1['Split Ratio'] = base_tmp1.apply(substituir_split_ratio, axis=1)
substituicoes = (split_ratio_antes != base_tmp1['Split Ratio']).sum()

print(f"Número de substituições feitas na coluna 'Split Ratio': {substituicoes}")

Número de substituições feitas na coluna 'Split Ratio': 54848


---

-> Agora, verificaremos se ainda há dados não padronizados nessa coluna

In [48]:
# Função para verificar se o valor está no formato correto
def formato_split_ratio_invalido(valor):
    if pd.isnull(valor):
        return True  # Nulos são considerados inválidos
    return not bool(re.match(r'^\d+:\d+$', str(valor).strip()))

# Aplica a função para identificar os inválidos
filtro_invalidos = base_tmp1['Split Ratio'].apply(formato_split_ratio_invalido)

# Conta quantos são inválidos
quantidade_invalidos = filtro_invalidos.sum()

# Lista os valores inválidos únicos e suas ocorrências
valores_invalidos = base_tmp1.loc[filtro_invalidos, 'Split Ratio'].value_counts(dropna=False)

# Exibe os resultados
print(f"Quantidade de valores fora do padrão x:y: {quantidade_invalidos}\n")
print("Valores inválidos encontrados:")
print(valores_invalidos)

Quantidade de valores fora do padrão x:y: 0

Valores inválidos encontrados:
Series([], Name: count, dtype: int64)


---

-> Assim, podemos copiar todas as modificações realizadas na base temporária para a original

In [49]:
base1 = base_tmp1.copy()

#### **Visualização final da padronização e limpeza da Base 1**

In [50]:
base1.head()

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio
0,1,2023-09-03,META,909.63,1380.05,258.44,595.86,61.54,8846040350,3.57,1:1
1,2,2024-08-28,NVDA,678.96,725.03,185.51,295.83,843.78,7346935699,2.85,3:2
2,3,2022-05-10,INTU,831.86,1348.94,215.00,874.05,336.19,8027589419,2.28,2:1
3,4,2017-12-26,NVDA,508.65,1111.85,186.39,511.49,952.63,1939500000,4.51,3:2
4,5,2022-01-20,WORK,692.20,1425.94,340.75,378.58,1493.44,8984007687,1.93,3:2


---
### **Base stock_part2**

#### **Análise de dados faltantes**

-> Primeiro vamos verificar se existem valores nulos na nossa base

In [51]:
null_counts = base2.isnull().sum()

print(null_counts)

columns_with_nulls = null_counts[null_counts > 0]
print(columns_with_nulls)

ID                0
Sector            0
Industry          0
Exchange          0
Currency          0
MarketCap         0
PE Ratio          0
EPS               0
Next Earnings     0
Previous Close    0
Day Range         0
dtype: int64
Series([], dtype: int64)


-> Como a base 2 não possui nenhum valor nulo aparente, vamos continuar com o tratamento das colunas

---

#### **Análises de linhas duplicadas**

In [52]:
# Retorna o número de linhas duplicadas
base2.duplicated().sum()

np.int64(0)

-> Notamos que não haverá preocupação em excluir linhas duplicadas

---


#### **Funções generalizadas para facilitar o tratamento das colunas**

##### **Verifica_Categorias_Base:**

-> Essa função foi criada com um intuito bem interessante, a ideia dela era usar a biblioteca thefuzz para poder agrupar e juntar possiveis palavras que estavam na mesma categoria, porem estavam com gramatica errada, exemplo: Healthcare e realtcare

-> Com isso, ela foi usada tanto na 1 quanto na 2 base, conseguindo nos dar um guia nas colunas em que valores estariamos tratando, evitando se perder nos erros gramaticais

-> Esses grupos foram usados de diferentes formas, mas a ideia era apenas que ela conseguisse gerar um caminho de nomes a se seguir, evitando que usassemos palavras que não existiam nas colunas

In [53]:
def Verifica_Categorias_base(dataframe, categoria):

  if categoria not in dataframe.columns:
        print(f"Erro: a categoria '{categoria}' não foi encontrada no DataFrame.")
        return

  valores = dataframe[categoria].dropna().astype(str).str.strip().unique().tolist()#Aqui estou pegando todos  os nomes da lista Sector
  grupos = []

  while valores:
      base = valores.pop(0)
      grupo = [base]
      similares = []

      for v in valores:
          if fuzz.ratio(base, v) >= 60:#Pega apenas os valores mais proxios para comparação
              grupo.append(v)
              similares.append(v)

      # Remove os que já foram agrupados
      valores = [v for v in valores if v not in similares]
      grupos.append(grupo)

  # Exibe os grupos
  for i, grupo in enumerate(grupos, 1):
      print(f"Grupo {i}: {grupo}")

-> Essa função é um complemento da função de cima, ela tambem usa thefuzz mas ela ja aplica as mudanças das categorias, tais categorias precisam ser dadas pelo usuario por meio de categorias_corrertas

-> Categorias_corretas é uma lista que, de forma manual, é feita com os nomes que vieram da função Verifica_Categorias_base e foram analisadas por nos

In [54]:
def corrigir_coluna_categoria(dataframe, coluna, categorias_corretas, limite_score=80):
    if coluna not in dataframe.columns:#tratamento de erro
        print(f"Erro: a coluna '{coluna}' não existe no DataFrame.")
        return dataframe  # retorna sem alteração

    def corrigir(valor):#utiliza a biblioteca thefuzz para poder categorizar os valores em proximidade
        valor = str(valor).lower().strip()
        correspondencia, score = process.extractOne(valor, categorias_corretas)
        return correspondencia if score >= limite_score else valor

    dataframe[coluna] = dataframe[coluna].apply(corrigir)#Aqui é onde acontece a aplicação do tratamento propriamente dito, aplicando o algoritmo
    return dataframe

-> encontrar_simbolos e extrair_sufixos_unicos tem ideias parecidas, ambas tentam procurar nas colunas valores que são diferentes de inteiros paara assim termos uma ideia do que tratar em colunas como a MarketCap que tinha inteiros misturado com string e apenas estavamos precisando de inteiros

-> A diferença é sutil, encontrar_simbolos pega qualquer coisa diferente de um inteiro.
Já encontrar_simbolos vai atras de Letras que podem estar misturadas a coluna

In [55]:
def encontrar_simbolos(df, coluna):
    if coluna not in df.columns:
        print(f"Coluna '{coluna}' não encontrada.")
        return set()

    simbolos = set()
    valores = df[coluna].dropna().astype(str)

    for valor in valores:
        encontrados = re.findall(r'[^0-9\s]', valor)  # tudo que não for número nem espaço
        simbolos.update(encontrados)

    return simbolos

In [56]:
def extrair_sufixos_unicos(coluna):
    sufixos = set()

    for valor in coluna.dropna():
        # Extrai a parte não numérica (sufixo) após os números e pontos
        match = re.findall(r'(?:\d+\.?\d*\s*)([a-zA-Z]+)', str(valor))
        sufixos.update([m.strip() for m in match])

    return sorted(sufixos)

-> A função converter_market_cap apesar de contra intuitivo, não trata apenas MarketCap, como trata a Volume da base 1. A ideia dela é que os valores M, K, B etc.. que são encontrados por meio da extrair_sufixos_unicos tenham valores agregados

-> Com esses valores o nosso papel foi então fazer uma logica que consiga manter o padrao de tamanho dos inteiros da MarketCap(13 inteiros) e Volume(10 inteiros)

-> Desssa forma a função pega todos os valores da coluna e verifica se eles ja estao na zona ideal de tamanho.
- Caso nao estejam eles são mutiplicados pelo sufixo
- Apos a multiplicação é verificado se eles extrapolaram a zona ideal, estão na zona ideal ou a baixo
- Se estrapolou, fica com o valor antigo
- Se esta na zona ideala ou a baixo, fica com o valor multiplicado

In [57]:
def converter_market_cap(base, coluna):
    multiplicadores = {
        'K': 1000,
        'M': 1000000,
        'B': 1000000000,
        'b': 1000000000,
        'billion': 1000000000,
        'MK': 1000000,
        'Mb': 1000000
    }

    def aplica(valor):
        match = re.search(r'(\d+(?:\.\d+)?)\s*([a-zA-Z]+)', str(valor))
        if match:
            numero = float(match.group(1))
            sufixo = match.group(2)
            fator = multiplicadores.get(sufixo, 1)
            res_mult = numero * fator
            if numero >=1e10 or res_mult >= 1e10:
                return int(numero)
            else:
                return int(res_mult)
        else:
            return int(valor)

    base[coluna] = base[coluna].apply(aplica)
    return base


---
#### **Tratamento da coluna Sector e Industry**

---

-> Para essas colunas, percebemos que ao fazer o alinhamento do ID da base 1 com o ID da base 2, o Sector e Industry do Ticker correspondente estava inconsistente na grande maioria das linhas. Para correção disso, realizamos uma pesquisa de cada Ticker presente na base 1 e descrevemos seu Sector e Industry:

1. Meta Platforms (META)

   * Sector: Comunicação
   * Industry: Mídia e Serviços Interativos

2. NVIDIA Corporation (NVDA)

   * Sector: Tecnologia
   * Industry: Semicondutores

3. Intuit Inc. (INTU)

   * Sector: Tecnologia
   * Industry: Software – Aplicativos

4. Slack Technologies (WORK)

   * Sector: Tecnologia
   * Industry: Software – Aplicativos

5. Alphabet Inc. (GOOG)

   * Sector: Comunicação
   * Industry: Serviços de Internet

6. Tesla Inc. (TSLA)

   * Sector: Consumo Cíclico
   * Industry: Veículos Automotores

7. Netflix Inc. (NFLX)

   * Sector: Comunicação
   * Industry: Entretenimento

8. Adobe Inc. (ADBE)

   * Sector: Tecnologia
   * Industry: Software – Aplicativos

9. PayPal Holdings Inc. (PYPL)

   * Sector: Tecnologia
   * Industry: Serviços de Pagamento

10. Uber Technologies Inc. (UBER)

    * Sector: Serviços
    * Industry: Serviços Profissionais

11. Snap Inc. (SNAP)

    * Sector: Comunicação
    * Industry: Mídia e Serviços Interativos

12. Amazon.com Inc. (AMZN)

    * Sector: Consumo Cíclico
    * Industry: Varejo de Internet

13. Oracle Corporation (ORCL)

    * Sector: Tecnologia
    * Industry: Software – Infraestrutura

14. Advanced Micro Devices Inc. (AMD)

    * Sector: Tecnologia
    * Industry: Semicondutores

15. Salesforce Inc. (CRM)

    * Sector: Tecnologia
    * Industry: Software – Aplicativos

16. Pinterest Inc. (PINS)

    * Sector: Comunicação
    * Industry: Mídia e Serviços Interativos

17. Shopify Inc. (SHOP)

    * Sector: Tecnologia
    * Industry: Software – Aplicativos

18. Block Inc. (SQ)

    * Sector: Tecnologia
    * Industry: Serviços de Pagamento

19. Intel Corporation (INTC)

    * Sector: Tecnologia
    * Industry: Semicondutores

20. Alibaba Group Holding Ltd. (BABA)

    * Sector: Consumo Cíclico
    * Industry: Varejo de Internet

21. Microsoft Corporation (MSFT)

    * Sector: Tecnologia
    * Industry: Software – Infraestrutura

22. Apple Inc. (AAPL)

    * Sector: Tecnologia
    * Industry: Hardware de Computador

23. Zoom Video Communications Inc. (ZM)

    * Sector: Tecnologia
    * Industry: Software – Aplicativos

24. Spotify Technology S.A. (SPOT)

    * Sector: Comunicação
    * Industry: Entretenimento

25. Lyft Inc. (LYFT)

    * Sector: Serviços
    * Industry: Serviços Profissionais

26. International Business Machines Corporation (IBM)

    * Sector: Tecnologia
    * Industry: Serviços de TI

27. Twitter Inc. (TWTR)

    * Sector: Comunicação
    * Industry: Mídia e Serviços Interativos

28. VMware Inc. (VMW)

    * Sector: Tecnologia
    * Industry: Software – Infraestrutura

29. DocuSign Inc. (DOCU)

    * Sector: Tecnologia
    * Industry: Software – Aplicativos

30. Cisco Systems Inc. (CSCO)

    * Sector: Tecnologia
    * Industry: Equipamentos de Comunicação

---




-> Baseado nisso, realizaremos uma limpeza das bases Sector e Industry da base2 e adicionaremos as mesmas colunas com dos dados corrigidos na base1 em relação ao ticker da base 1 correspondente

In [58]:
# dicionário com setor e indústria corretos por yicker
referencia_ticker = {
    'META':   {'Sector': 'Communication', 'Industry': 'Media & Interactive Services'},
    'NVDA':   {'Sector': 'Technology', 'Industry': 'Semiconductors'},
    'INTU':   {'Sector': 'Technology', 'Industry': 'Software – Application'},
    'WORK':   {'Sector': 'Technology', 'Industry': 'Software – Application'},
    'GOOG':   {'Sector': 'Communication', 'Industry': 'Internet Services'},
    'TSLA':   {'Sector': 'Consumer Cyclical', 'Industry': 'Auto Manufacturers'},
    'NFLX':   {'Sector': 'Communication', 'Industry': 'Entertainment'},
    'ADBE':   {'Sector': 'Technology', 'Industry': 'Software – Application'},
    'PYPL':   {'Sector': 'Technology', 'Industry': 'Credit Services'},
    'UBER':   {'Sector': 'Services', 'Industry': 'Professional Services'},
    'SNAP':   {'Sector': 'Communication', 'Industry': 'Media & Interactive Services'},
    'AMZN':   {'Sector': 'Consumer Cyclical', 'Industry': 'Internet Retail'},
    'ORCL':   {'Sector': 'Technology', 'Industry': 'Software – Infrastructure'},
    'AMD':    {'Sector': 'Technology', 'Industry': 'Semiconductors'},
    'CRM':    {'Sector': 'Technology', 'Industry': 'Software – Application'},
    'PINS':   {'Sector': 'Communication', 'Industry': 'Media & Interactive Services'},
    'SHOP':   {'Sector': 'Technology', 'Industry': 'Software – Application'},
    'SQ':     {'Sector': 'Technology', 'Industry': 'Credit Services'},
    'INTC':   {'Sector': 'Technology', 'Industry': 'Semiconductors'},
    'BABA':   {'Sector': 'Consumer Cyclical', 'Industry': 'Internet Retail'},
    'MSFT':   {'Sector': 'Technology', 'Industry': 'Software – Infrastructure'},
    'AAPL':   {'Sector': 'Technology', 'Industry': 'Computer Hardware'},
    'ZM':     {'Sector': 'Technology', 'Industry': 'Software – Application'},
    'SPOT':   {'Sector': 'Communication', 'Industry': 'Entertainment'},
    'LYFT':   {'Sector': 'Services', 'Industry': 'Professional Services'},
    'IBM':    {'Sector': 'Technology', 'Industry': 'Information Technology Services'},
    'TWTR':   {'Sector': 'Communication', 'Industry': 'Media & Interactive Services'},
    'VMW':    {'Sector': 'Technology', 'Industry': 'Software – Infrastructure'},
    'DOCU':   {'Sector': 'Technology', 'Industry': 'Software – Application'},
    'CSCO':   {'Sector': 'Technology', 'Industry': 'Communication Equipment'}
}


base_temppp = base1.copy()

base2.drop(columns=['Sector', 'Industry'], errors='ignore', inplace=True)

# Função para recuperar setor e indústria com base no ticker
def obter_sector_industry(ticker):
    if ticker in referencia_ticker:
        return pd.Series([referencia_ticker[ticker]['Sector'], referencia_ticker[ticker]['Industry']])
    else:
        return pd.Series([None, None])  # Caso o ticker não esteja no dicionário

# Aplica a função
base_temppp[['Sector', 'Industry']] = base_temppp['Ticker'].apply(obter_sector_industry)

# Exibe algumas linhas para verificação
base_temppp[['Ticker', 'Sector', 'Industry']].tail()


,Ticker,Sector,Industry
199995,TSLA,Consumer Cyclical,Auto Manufacturers
199996,VMW,Technology,Software – Infrastructure
199997,AAPL,Technology,Computer Hardware
199998,DOCU,Technology,Software – Application
199999,CRM,Technology,Software – Application


---

-> Agora passaremos essas mudanças para a base original

In [59]:
base1 = base_temppp.copy()

---
#### **Tratamento da coluna Exchange**

-> Nessas colunas, o nosso desafio é simples, mas ao mesmo tempo complicado: temos basicamente os grupos que precisamos padronizar.

-> No entanto, eles apresentam erros de ortografia. Para contornar essa situação e padronizar os valores, utilizaremos a biblioteca thefuzz, que permite identificar e agrupar nomes semelhantes com base em similaridade textual.

In [60]:
# Copia da base2 para garantir integridade da base enquando estou executando o tratamento
base2_temp = base2.copy()

In [61]:
Verifica_Categorias_base(base2_temp, 'Exchange')

Grupo 1: ['NYSE', 'NYQE', 'NYSZ', 'NFSE', 'NYSC', 'NTSE', 'SYSE', 'NYSA', 'NYFE', 'NYWE', 'NYTE', 'NYSP', 'NHSE', 'NYBE', 'NUSE', 'NYSU', 'NYSF', 'QYSE', 'MYSE', 'EYSE', 'IYSE', 'NYSL', 'NYJE', 'NYKE', 'NYOE', 'NYSD', 'NKSE', 'HYSE', 'NYSK', 'TYSE', 'NDSE', 'NASEAQ', 'NYAE', 'LYSE', 'KYSE', 'CYSE', 'NZSE', 'UYSE', 'NYGE', 'NYUE', 'NQSE', 'NYSJ', 'RYSE', 'NYST', 'NYNE', 'NBSE', 'NYSI', 'NYEE', 'BYSE', 'NVSE', 'NYXE', 'NRSE', 'XYSE', 'NSE', 'NYLE', 'NLSE', 'GYSE', 'NYME', 'YSE', 'NYHE', 'DYSE', 'YYSE', 'NYIE', 'NYZE', 'NCSE', 'NNSE', 'NESE', 'NASE', 'NYSB', 'NYSDAQ', 'NYSM', 'NYSH', 'NWSE', 'NYVE', 'NJSE', 'NYSN', 'NYSX', 'PYSE', 'NYSS', 'NGSE', 'NOSE', 'NYSY', 'AYSE', 'NYSQ', 'FYSE', 'WYSE', 'NYPE', 'NYSR', 'NSSE', 'NYCE', 'NMSE', 'NYRE', 'NYSW', 'NYSV', 'NASDAE', 'ZYSE', 'NASDEQ', 'NYSG', 'NYDE', 'JYSE', 'NYYE', 'NYSO', 'NXSE', 'NISE', 'VYSE', 'NPSE', 'OYSE']
Grupo 2: ['TSE', 'LSE', 'TSX', 'SSE', 'VSE', 'TSQ', 'ISE', 'SFE', 'SGE', 'QSE', 'TSC', 'GSE', 'TSB', 'HSE', 'USE', 'TBE', 'TSJ',

-> Aqui corrijimos os problemas de padronização passando tudo para o mesmo formato de texto

In [62]:
exchange_dicionario = {
    "NYSE": "NYSE",
    "NASDAQ": "NASDAQ",
    "LSE": "LSE",
    "HKEX": "HKEX",
    "TSE": "TSE",
    "EURONEXT": "EURONEXT",
    "TSX": "TSX",
    "SSE": "SSE",
    "SGX": "SGX",
    "ASX": "ASX",
    "AMEX": "AMEX",

    "nyse": "NYSE",
    "nasdaq": "NASDAQ",
    "lse": "LSE",
    "hkex": "HKEX",
    "tse": "TSE",
    "euronext": "EURONEXT",
    "tsx": "TSX",
    "sse": "SSE",
    "sgx": "SGX",
    "asx": "ASX",
    "amex": "AMEX"
}
base2_temp['Exchange'] = base2_temp['Exchange'].str.upper()
base2_temp['Exchange'] = base2_temp['Exchange'].replace(exchange_dicionario)

print(base2_temp['Exchange'].head(10))

0      NYSE
1       TSE
2       LSE
3      HKEX
4    NASDAQ
5       TSX
6    NASDAQ
7       TSX
8      AMEX
9      AMEX
Name: Exchange, dtype: object


-> Criamos uma tabela para que seja possível ver todas as siglas presentes na base e quantas vezes cada uma aparece. Cada sigla principal aparece cerca de 20.000 vezes, já os erros ortográficos possuem frequências consideravelmente menores

In [63]:
base2_temp['Exchange'] = base2_temp['Exchange'].str.strip()

variacoes_siglas = base2_temp['Exchange'].value_counts().reset_index()
variacoes_siglas.columns = ['variacao', 'frequencia']

#Para que possamos ver todas as siglas e sua frequência
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print(variacoes_siglas)

     variacao  frequencia
0        HKEX       21285
1         TSX       21277
2         SSE       21270
3    EURONEXT       21196
4        NYSE       21176
5      NASDAQ       21150
6        AMEX       21130
7         TSE       21047
8         LSE       20912
9         DSE          51
10        YSE          51
11        OSE          51
12        PSE          49
13        NSE          47
14        VSE          47
15        JSE          47
16        GSE          47
17        HSE          47
18        USE          46
19        ASE          45
20        ISE          44
21        RSE          42
22        QSE          42
23        BSE          41
24        TSO          40
25        CSE          40
26        TSL          39
27        ZSE          39
28        WSE          39
29        MSE          39
30        XSE          39
31        TSJ          36
32        ESE          36
33        KSE          35
34        TSM          34
35        TSN          34
36        TSW          33
37        TS

-> Alocando as siglas parecidas em grupos, usando a biblioteca thefuzz, para que fique mais fácil corrijir os erros de cada grupo

In [64]:
from thefuzz import fuzz

#Utilizando a função criada por Jairo com adição do retorno agora, uma vez que manipularemos os grupos
def Verifica_Categorias_base_retorno(dataframe, categoria):

    if categoria not in dataframe.columns:
        print(f"Erro: a categoria '{categoria}' não foi encontrada no DataFrame.")
        return []

    valores = dataframe[categoria].dropna().astype(str).str.strip().unique().tolist()
    grupos = []

    while valores:
        base = valores.pop(0)
        grupo = [base]
        similares = []

        for v in valores:
            if fuzz.ratio(base, v) >= 60:  # Pega apenas os valores mais próximos para comparação
                grupo.append(v)
                similares.append(v)

        # Remove os que já foram agrupados
        valores = [v for v in valores if v not in similares]
        grupos.append(grupo)

    # Exibe os grupos
    for i, grupo in enumerate(grupos, 1):
        print(f"Grupo {i}: {grupo}")

    return grupos  # <-- retorno adicionado aqui


grupos_siglas_b2 = Verifica_Categorias_base_retorno(variacoes_siglas, 'variacao')

Grupo 1: ['HKEX', 'AKEX', 'HMEX', 'HKEK', 'HKEQ', 'XKEX', 'HKDX', 'HKEN', 'HKEB', 'HCEX', 'HKEP', 'HKEF', 'HKHX', 'HKJX', 'HKES', 'HKEC', 'HWEX', 'HDEX', 'MKEX', 'HKEV', 'HKQX', 'HGEX', 'QKEX', 'HKYX', 'KMEX', 'HKCX', 'HKNX', 'HKED', 'HTEX', 'FKEX', 'HKEE', 'HKET', 'HLEX', 'HKGX', 'HKIX', 'HHEX', 'HAEX', 'HKEH', 'HKER', 'HSEX', 'HZEX', 'IKEX', 'HKBX', 'HKTX', 'HKEL', 'TKEX', 'HKSX', 'HOEX', 'GKEX', 'HKAX', 'HKEO', 'HPEX', 'HEEX', 'AHEX', 'HQEX', 'HKEG', 'HKEA', 'HKKX', 'HNEX', 'HKRX', 'BKEX', 'DKEX', 'HKEI', 'EKEX', 'HKEM', 'NKEX', 'ZKEX', 'SKEX', 'HFEX', 'VKEX', 'RKEX', 'YKEX', 'HKFX', 'HYEX', 'HKZX', 'KKEX', 'HREX', 'HJEX', 'PKEX', 'HKWX', 'HVEX', 'WKEX', 'HKOX', 'HIEX', 'HKEZ', 'HKEW', 'HKEY', 'HKEJ', 'UKEX', 'HKUX', 'HKEU', 'CKEX', 'OKEX', 'HBEX', 'HUEX', 'HKMX', 'HKPX', 'HKXX', 'HKLX', 'HKVX', 'LKEX', 'JKEX', 'HXEX']
Grupo 2: ['TSX', 'TSE', 'TSO', 'TSL', 'TSJ', 'TSM', 'TSN', 'TSW', 'TSI', 'TSQ', 'TSK', 'TSH', 'TSC', 'PSX', 'LSX', 'TSB', 'TSV', 'TSA', 'TSZ', 'TSF', 'TSS', 'TSG', 'T

-> Corrijindo problemas manualmente

In [65]:
grupos_siglas_b2[9].extend(['LSE', 'LSS', 'LQE', 'LSL', 'LSU', 'LSQ', 'LSF', 'LSK', 'LSZ', 'LSR', 'LSI', 'LSV', 'LSY', 'LSM', 'LSG', 'LSW', 'LSJ', 'LSO', 'LST', 'LSA', 'LSD', 'LSN', 'LSB', 'LSC', 'LSP', 'LLE', 'LSH'])

grupos_siglas_b2.pop(8)

['LSL',
 'LSU',
 'LSQ',
 'LSF',
 'LSK',
 'LSZ',
 'LSR',
 'LSI',
 'LSV',
 'LSY',
 'LSM',
 'LSG',
 'LSW',
 'LSJ',
 'LSO',
 'LST',
 'LSA',
 'LSD',
 'LSN',
 'LSB',
 'LSC',
 'LSP',
 'LLE',
 'LSH']

-> Criando dicionários com os grupos todos montados para que seja possível a correção das siglas de maneira correta para cada bolsa de valores que elas representam

In [66]:
# Função para corrigir o valor com base no grupo
def corrigir_exchange(valor):
    if not isinstance(valor, str):
        return valor  # Retorna o valor original se não for string

    valor = valor.strip().upper()  # Remover espaços e converter para maiúsculas

    # Mapeamento dos grupos com suas variações (baseado na sua análise)
    grupos = {
        'HKEX': ['HKEX', 'AKEX', 'HMEX', 'HKEK', 'HKEQ', 'XKEX', 'HKDX', 'HKEN', 'HKEB', 'HCEX', 'HKEP', 'HKEF', 'HKHX', 'HKJX', 'HKES', 'HKEC', 'HWEX', 'HDEX', 'MKEX', 'HKEV', 'HKQX', 'HGEX', 'QKEX', 'HKYX', 'KMEX', 'HKCX', 'HKNX', 'HKED', 'HTEX', 'FKEX', 'HKEE', 'HKET', 'HLEX', 'HKGX', 'HKIX', 'HHEX', 'HAEX', 'HKEH', 'HKER', 'HSEX', 'HZEX', 'IKEX', 'HKBX', 'HKTX', 'HKEL', 'TKEX', 'HKSX', 'HOEX', 'GKEX', 'HKAX', 'HKEO', 'HPEX', 'HEEX', 'AHEX', 'HQEX', 'HKEG', 'HKEA', 'HKKX', 'HNEX', 'HKRX', 'BKEX', 'DKEX', 'HKEI', 'EKEX', 'HKEM', 'NKEX', 'ZKEX', 'SKEX', 'HFEX', 'VKEX', 'RKEX', 'YKEX', 'HKFX', 'HYEX', 'HKZX', 'KKEX', 'HREX', 'HJEX', 'PKEX', 'HKWX', 'HVEX', 'WKEX', 'HKOX', 'HIEX', 'HKEZ', 'HKEW', 'HKEY', 'HKEJ', 'UKEX', 'HKUX', 'HKEU', 'CKEX', 'OKEX', 'HBEX', 'HUEX', 'HKMX', 'HKPX', 'HKXX', 'HKLX', 'HKVX', 'LKEX', 'JKEX', 'HXEX'],
        'TSX': ['TSX', 'TSO', 'TSL', 'TSJ', 'TSM', 'TSN', 'TSW', 'TSI', 'TSQ', 'TSK', 'TSH', 'TSC', 'PSX', 'LSX', 'TSB', 'TSV', 'TSA', 'TSZ', 'TSF', 'TSS', 'TSG', 'TSD', 'TST', 'TSY', 'TSR', 'SSX', 'TSP', 'TUX', 'CSX', 'TSU', 'TGX', 'BSX', 'TKX', 'TLX', 'TXE', 'TWX', 'TYX', 'ESX', 'TFX', 'TJX', 'ISX', 'SXE', 'XSX', 'FSX', 'GSX', 'USX', 'TXX', 'TEX', 'TAX', 'JSX', 'RSX', 'TPX', 'TVX', 'TNX', 'TMX', 'TZX', 'TDX', 'TTX', 'HSX', 'TBX', 'TRX', 'DSX', 'VSX', 'TIX', 'TOX', 'WSX', 'QSX', 'ZSX', 'YSX', 'OSX', 'ASX', 'TCX', 'THX', 'MSX', 'NSX', 'TQX', 'KSX'],
        'SSE': ['SSE', 'DSE', 'YSE', 'OSE', 'PSE', 'NSE', 'VSE', 'JSE', 'GSE', 'HSE', 'USE', 'ASE', 'ISE', 'RSE', 'QSE', 'BSE', 'CSE', 'ZSE', 'WSE', 'MSE', 'XSE', 'ESE', 'KSE', 'FSE', 'SUE', 'SSV', 'SWE', 'SJE', 'SBE', 'SHE', 'SSB', 'SCE', 'SYSE', 'SNE', 'SSL', 'SKE', 'SRE', 'SSN', 'SZE', 'SSD', 'SSJ', 'SSO', 'SGE', 'SSQ', 'SSS', 'SAE', 'SYE', 'SPE', 'SSG', 'SFE', 'SSY', 'SSM', 'SST', 'SIE', 'SSF', 'SSW', 'SSK', 'SOE', 'SSZ', 'SEE', 'SSU', 'SLE', 'SSH', 'NSSE', 'SSA', 'SQE', 'SDE', 'SSC', 'SVE', 'SSP', 'SSR', 'SSI', 'STE', 'SME'],
        'EURONEXT': ['EURONEXT', 'EUAONEXT', 'EUROOEXT', 'BURONEXT', 'EUROIEXT', 'EARONEXT', 'EURONFXT', 'EURONEZT', 'EURUNEXT', 'EURONEXN', 'EUYONEXT', 'MURONEXT', 'EUROKEXT', 'EUWONEXT', 'EURZNEXT', 'EURONEXR', 'EUUONEXT', 'EUOONEXT', 'UURONEXT', 'EURONWXT', 'EURONNXT', 'EURONEXZ', 'EURONECT', 'EGRONEXT', 'EUROEEXT', 'EURBNEXT', 'EURXNEXT', 'EURONEST', 'EURONEXG', 'EURONEPT', 'EURYNEXT', 'EUROWEXT', 'EHRONEXT', 'EURGNEXT', 'EXRONEXT', 'EURONEJT', 'EURONEAT', 'EURONEXY', 'EUROLEXT', 'CURONEXT', 'EURONVXT', 'LURONEXT', 'EUROMEXT', 'NURONEXT', 'EURWNEXT', 'DURONEXT', 'EURONEFT', 'EURONKXT', 'EURNNEXT', 'EPRONEXT', 'EURQNEXT', 'EURONEUT', 'EURONEVT', 'EUROHEXT', 'EURONBXT', 'EURONEXU', 'EURONGXT', 'EURTNEXT', 'EURONLXT', 'EURODEXT', 'EURENEXT', 'XURONEXT', 'EUROTEXT', 'EURONEXX', 'ELRONEXT', 'ZURONEXT', 'EUROXEXT', 'EURONOXT', 'RURONEXT', 'EURONERT', 'EURVNEXT', 'EUROJEXT', 'EUROPEXT', 'EULONEXT', 'EUROYEXT', 'EURONDXT', 'EURONEXB', 'EURONEDT', 'EUROGEXT', 'EUROUEXT', 'EMRONEXT', 'EUSONEXT', 'EURONEET', 'EERONEXT', 'EURFNEXT', 'EURONENT', 'ETRONEXT', 'EUKONEXT', 'EURONEHT', 'FURONEXT', 'EUROCEXT', 'EURONJXT', 'EURONHXT', 'EJRONEXT', 'EUROZEXT', 'ECRONEXT', 'EURONPXT', 'EUROSEXT', 'EBRONEXT', 'EUCONEXT', 'EURONYXT', 'EURONEXD', 'EVRONEXT', 'EUMONEXT', 'ESRONEXT', 'EURJNEXT', 'EURPNEXT', 'EURONRXT', 'EURONEXS', 'EUDONEXT', 'EURMNEXT', 'SURONEXT', 'EURONEXW', 'EUTONEXT', 'EUBONEXT', 'EURONEBT', 'EUQONEXT', 'EUJONEXT', 'EURONETT', 'EFRONEXT', 'JURONEXT', 'EUVONEXT', 'EUEONEXT', 'EURONCXT', 'EURONEXA', 'PURONEXT', 'EURONZXT', 'EURONEXL', 'EUZONEXT', 'EURSNEXT', 'GURONEXT', 'EURONEXQ', 'EURONEXO', 'EURONEYT', 'EURONEKT', 'EURONAXT', 'EURONEXJ', 'EURONQXT', 'EUGONEXT', 'EURONEXE', 'KURONEXT', 'EURRNEXT', 'IURONEXT', 'EURANEXT', 'EURONXXT', 'VURONEXT', 'EURONEXI', 'EZRONEXT', 'EUROQEXT', 'EURHNEXT', 'EUPONEXT', 'EUIONEXT', 'EURONUXT', 'EIRONEXT', 'EDRONEXT', 'EWRONEXT', 'EQRONEXT', 'EUROFEXT', 'ENRONEXT', 'EURONEMT', 'EUROAEXT', 'EKRONEXT', 'EURONEQT', 'EUHONEXT', 'EURONEXV', 'TURONEXT', 'EUNONEXT', 'HURONEXT', 'EURONTXT', 'EURCNEXT', 'EURDNEXT', 'EURINEXT', 'AURONEXT', 'EURLNEXT', 'QURONEXT', 'EURONIXT', 'EORONEXT', 'EURONEXP', 'EUROBEXT', 'EUXONEXT', 'EURONEXK', 'EURONMXT', 'WURONEXT', 'EURONEWT', 'EURONEXM', 'EURONEXF', 'EURONEXH', 'EURONEOT', 'YURONEXT', 'EYRONEXT', 'EUROREXT', 'EURKNEXT', 'EUROVEXT', 'ERRONEXT', 'EURONELT', 'EURONEIT', 'EURONEXC', 'OURONEXT', 'EUFONEXT', 'EURONEGT', 'EURONSXT'],
        'NYSE': ['NYSE', 'NLSE', 'NYQE', 'AYSE', 'NYSK', 'XYSE', 'NTSE', 'NYSM', 'NYVE', 'MYSE', 'NKSE', 'NRSE', 'NYCE', 'NYIE', 'NYXE', 'CYSE', 'NYSH', 'NUSE', 'NYSP', 'NYJE', 'NQSE', 'NVSE', 'NYSL', 'NYSF', 'NYSC', 'NYST', 'FYSE', 'NWSE', 'NYBE', 'NYSA', 'NYRE', 'QYSE', 'NYZE', 'NYFE', 'NYSI', 'NBSE', 'NYKE', 'HYSE', 'PYSE', 'GYSE', 'NYSR', 'NNSE', 'NDSE', 'NYSD', 'NYNE', 'RYSE', 'NYSJ', 'NYPE', 'NYSW', 'NYTE', 'NYWE', 'YYSE', 'LYSE', 'NYHE', 'NGSE', 'NOSE', 'EYSE', 'NYSY', 'OYSE', 'NYSV', 'NYEE', 'NCSE', 'BYSE', 'NZSE', 'NYLE', 'DYSE', 'NYSG', 'NXSE', 'NYSZ', 'NYSB', 'NYSQ', 'NFSE', 'NYGE', 'NYSX', 'UYSE', 'IYSE', 'NMSE', 'NYSU', 'ZYSE', 'NHSE', 'NYYE', 'NYDE', 'NPSE', 'TYSE', 'NASE', 'NYAE', 'NESE', 'NYUE', 'NYOE', 'NYSS', 'NISE', 'WYSE', 'NYME', 'NJSE', 'NYSO', 'NYSN', 'KYSE', 'JYSE', 'VYSE'],
        'NASDAQ': ['NASDAQ', 'NASNAQ', 'NADDAQ', 'NZSDAQ', 'NASDTQ', 'NDSDAQ', 'JASDAQ', 'NASDUQ', 'NASDAB', 'NNSDAQ', 'NMSDAQ', 'FASDAQ', 'NAQDAQ', 'GASDAQ', 'NASDNQ', 'NHSDAQ', 'NASDOQ', 'NANDAQ', 'QASDAQ', 'NAVDAQ', 'NASDRQ', 'OASDAQ', 'NASDMQ', 'NATDAQ', 'NAYDAQ', 'BASDAQ', 'NASDWQ', 'NASXAQ', 'NAIDAQ', 'NAJDAQ', 'MASDAQ', 'NASDHQ', 'NASDAR', 'NAHDAQ', 'NQSDAQ', 'NASDAZ', 'NALDAQ', 'NWSDAQ', 'NAXDAQ', 'NISDAQ', 'NASDAQ', 'NASDJQ', 'NPSDAQ', 'NKSDAQ', 'NASDIQ', 'AASDAQ', 'NAEDAQ', 'NFSDAQ', 'NASDAV', 'NASDBQ', 'NOSDAQ', 'NASDAJ', 'KASDAQ', 'NASKAQ', 'NASDAH', 'NASDPQ', 'HASDAQ', 'NASDAP', 'NASDVQ', 'NASDFQ', 'YASDAQ', 'NAGDAQ', 'ZASDAQ', 'NASDAO', 'NASDDQ', 'NASDAN', 'RASDAQ', 'TASDAQ', 'NASLAQ', 'NAMDAQ', 'SASDAQ', 'NASUAQ', 'NASHAQ', 'NASDAF', 'NASSAQ', 'NASRAQ', 'NASDAD', 'NASBAQ', 'NTSDAQ', 'NRSDAQ', 'NASDUQ'],
        'AMEX': ['AMEX', 'AMEU', 'AMYX', 'AMVX', 'ADEX', 'AMEH', 'AMET', 'VMEX', 'AMAX', 'WMEX', 'AMNX', 'AMEY', 'AMEL', 'AWEX', 'AMER', 'XMEX', 'AMEW', 'ANEX', 'AMWX', 'AMEO', 'AAEX', 'OMEX', 'EMEX', 'JMEX', 'AMEP', 'AMCX', 'AMEB', 'AMEF', 'AMEZ', 'AMMX', 'AMIX', 'AMHX', 'AMEG', 'CMEX', 'AMEA', 'UMEX', 'QMEX', 'FMEX', 'AREX', 'AMUX', 'AMTX', 'AJEX', 'AMEC', 'AIEX', 'AMEK', 'AXEX', 'ATEX', 'AMEM', 'ZMEX', 'GMEX', 'AMEE', 'AMFX', 'AMRX', 'AMEN', 'AFEX', 'LMEX', 'PMEX', 'NMEX', 'AMQX', 'AYEX', 'TMEX', 'AMPX', 'YMEX', 'AMOX', 'ALEX', 'AMDX', 'AMBX', 'AMEJ', 'AMES', 'AOEX', 'AMJX', 'SMEX', 'AQEX', 'DMEX', 'AMLX', 'AUEX', 'AEEX', 'AMGX', 'AMSX', 'AMXX', 'ACEX', 'AMEI', 'APEX', 'AMZX', 'AZEX', 'AMKX', 'AVEX', 'MMEX', 'AMEV', 'AMEQ', 'RMEX', 'ABEX', 'AGEX', 'IMEX', 'BMEX', 'ASEX', 'AMED'],
        'TSE': ['TQE', 'TIE', 'TPE', 'TVE', 'TAE', 'TWE', 'TEE', 'TCE', 'TBE', 'TUE', 'TGE', 'TFE', 'TKE', 'TJE', 'TTE', 'TOE', 'TLE', 'THE', 'TRE', 'TME', 'TYE', 'TDE', 'TNE', 'TZE','TSE'],
        'LSE': ['LSE', 'LOE', 'LHE', 'LCE', 'LVE', 'LUE', 'LPE', 'LRE', 'LYE', 'LNE', 'LJE', 'LDE', 'LIE', 'LME', 'LBE', 'LZE', 'LWE', 'LEE', 'LGE', 'LFE', 'LKE', 'LAE', 'LXE', 'LSL', 'LSU', 'LSQ', 'LSF', 'LSK', 'LSZ', 'LSR', 'LSI', 'LSV', 'LSY', 'LSM', 'LSG', 'LSW', 'LSJ', 'LSO', 'LST', 'LSA', 'LSD', 'LSN', 'LSB', 'LSC', 'LSP', 'LLE', 'LSH', 'LQE', 'LSS', 'LTE']
    }

    # Verificar se o valor está em algum grupo
    for chave_grupo, variacoes in grupos.items():
        if valor in variacoes:
            return chave_grupo.upper()

    # Se não encontrar, retorna o valor original
    return valor

    # Aplicar a função à coluna exchange (índice 3)
base2_temp['Exchange'] = base2_temp['Exchange'].apply(corrigir_exchange)

-> Fazendo a checagem, é possível perceber que ainda existem variações, então vamos corrijí-las

In [67]:
variacoes_siglas = base2_temp['Exchange'].value_counts().reset_index()
variacoes_siglas.columns = ['variacao', 'frequencia']

#Para que possamos ver todas as siglas e sua frequência
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print(variacoes_siglas)

    variacao  frequencia
0        SSE       22991
1        TSX       22804
2       HKEX       22381
3   EURONEXT       22303
4       NYSE       22199
5       AMEX       22164
6     NASDAQ       21812
7        LSE       21577
8        TSE       21398
9     NARDAQ           9
10    NASGAQ           9
11    VASDAQ           9
12    NAUDAQ           9
13    EASDAQ           8
14    NASMAQ           8
15    NASDEQ           8
16    NASDLQ           7
17    NUSDAQ           7
18    NASDAU           7
19    NASFAQ           7
20    NASDZQ           7
21    NJSDAQ           6
22    WASDAQ           6
23    NASDAM           6
24    NASDAX           6
25    NASDAE           6
26    NASPAQ           6
27    NVSDAQ           6
28    NASDXQ           6
29    NASDAW           6
30    NABDAQ           6
31    NBSDAQ           6
32    NAKDAQ           6
33    NASDAI           6
34    NXSDAQ           5
35    CASDAQ           5
36    NASWAQ           5
37    NASCAQ           5
38    NASTAQ           5


-> Realizando as últimas correções na coluna para que a padronização fique correta. Verficação final mostrando que a coluna foi tratada com sucesso

In [68]:
# Dicionário com as variações incorretas de NASDAQ
correcoes_nasdaq = {
    'NARDAQ', 'NASGAQ', 'VASDAQ', 'NAUDAQ', 'EASDAQ', 'NASMAQ', 'NASDEQ',
    'NASDLQ', 'NUSDAQ', 'NASDAU', 'NASFAQ', 'NASDZQ', 'NJSDAQ', 'WASDAQ',
    'NASDAM', 'NASDAX', 'NASDAE', 'NASPAQ', 'NVSDAQ', 'NASDXQ', 'NASDAW',
    'NABDAQ', 'NBSDAQ', 'NAKDAQ', 'NASDAI', 'NXSDAQ', 'CASDAQ', 'NASWAQ',
    'NASCAQ', 'NASTAQ', 'NACDAQ', 'NASDCQ', 'NASQAQ', 'UASDAQ', 'NESDAQ',
    'NASDAY', 'NASYAQ', 'NASDYQ', 'NAWDAQ', 'NASEAQ', 'NCSDAQ', 'NASDAL',
    'NASDAA', 'NASDQQ', 'LASDAQ', 'NASDAK', 'NAZDAQ', 'NASIAQ', 'NASDAG',
    'NAODAQ', 'DASDAQ', 'NAFDAQ', 'NASDSQ', 'PASDAQ', 'NASDAS', 'NYSDAQ',
    'NAADAQ', 'NGSDAQ', 'NSSDAQ', 'NASVAQ', 'NASDAT', 'NASDKQ', 'NAPDAQ',
    'NASOAQ', 'NASJAQ', 'NASDAC', 'NASDGQ', 'NASZAQ', 'NLSDAQ', 'XASDAQ',
    'NASAAQ', 'IASDAQ'
}

# Função para corrigir a coluna exchange (índice 3)
def corrigir_nasdaq(valor):
    if isinstance(valor, str) and valor.upper() in correcoes_nasdaq:
        return 'NASDAQ'
    return valor

# Aplicar a função à coluna exchange (índice 3)
base2_temp['Exchange'] = base2_temp['Exchange'].apply(corrigir_nasdaq)

print(base2_temp['Exchange'].unique())

['NYSE' 'TSE' 'LSE' 'HKEX' 'NASDAQ' 'TSX' 'AMEX' 'SSE' 'EURONEXT']


---
#### **Tratamento da coluna Currency**

-> Essa coluna é afetada por vários erros de formatação e digitação, por isso vamos criar um dicionáio com as formas incorretas para padronizar

In [69]:
currency_dicionario = {
    "USD": "USD",
    "US Dollar": "USD",
    "$": "USD",
    "usd": "USD",
    "US DOLLAR": "USD",
    "US DollaS": "USD",
    "US DBLLAR": "USD",
    "US DXllar": "USD",
    "US uollar": "USD",
    "US DolEar": "USD",
    "UE Dollar": "USD",
    "US DollaJ": "USD",
    "USR": "USD",
    "USRDollar": "USD",
}

base2_temp['Currency'] = base2_temp['Currency'].replace(currency_dicionario)

print(base2_temp[['Currency']].head(10))

  Currency
0      USD
1      USD
2      USD
3      USD
4      USD
5      USD
6      USD
7      USD
8      USD
9      USD


-> Analisando os valores únicos da coluna, ainda existem muitos erros de digitação que devem ser corrigidos, para isso vamos criar outro dicionário contendo todos esses erros e passando eles para a formatação correta

In [70]:
valores_unicos = base2_temp['Currency'].unique()

print(valores_unicos)

['USD' 'us dollar' 'US Vollar' 'WS Dollar' 'RSD' 'LSD' 'YSD' 'UID' 'ASD'
 'US DollGr' 'US DollVr' 'USN' 'HSD' 'US DKllar' 'US DollTr' 'UX Dollar'
 'US DoMlar' 'UY Dollar' 'US DOLYAR' 'US Jollar' 'US DolNar' 'USKDollar'
 'Isd' 'UH Dollar' 'US DollaT' 'ISD' 'US DAllar' 'usG' 'US DoOlar' 'USV'
 'usT' 'US DollYr' 'US DOLLOR' 'KSD' 'UK Dollar' 'US Aollar' 'US DollHr'
 'US DolBar' 'ZSD' 'USX' 'US DoUlar' 'MSD' 'MS Dollar' 'US Uollar'
 'US Xollar' 'USP' 'KS Dollar' 'USL' 'US DolFar' 'UOD' 'UZD' 'US DolVar'
 'Csd' 'USB' 'US DoJlar' 'PSD' 'US DoYlar' 'UAD' 'USU' 'YS Dollar'
 'IS Dollar' 'XSD' 'US DoAlar' 'Rs dollar' 'US DOLLTR' 'UPD' 'UR Dollar'
 'QS DOLLAR' 'UED' 'US DollaZ' 'US DollaV' 'USK' 'US DYllar' 'US DLllar'
 'USZ' 'US Follar' 'UQD' 'LS Dollar' 'AS Dollar' 'us dollRr' 'USH'
 'US DOLKAR' 'US DoPlar' 'USJ' 'UKD' 'US DollLr' 'US DollaY' 'XS Dollar'
 'VSD' 'US DolLar' 'CSD' 'US Eollar' 'US DoLlar' 'US DoNlar' 'us dollaR'
 'UE DOLLAR' 'DSD' 'UBD' 'US DollNr' 'SSD' 'VS Dollar' 'US DoDlar' 'U

In [71]:
currency_dicionario2 = {
    "us dollar": "USD",
    "US Dollar": "USD",
    "$": "USD",
    "usd": "USD",
    "USD": "USD",
    "US DOLLAR": "USD",
    "US DollaS": "USD",
    "US DBLLAR": "USD",
    "US DXllar": "USD",
    "US uollar": "USD",
    "US DolEar": "USD",
    "UE Dollar": "USD",
}

for variation in [
    'us dollar', 'US Vollar', 'WS Dollar', 'RSD', 'LSD', 'YSD', 'UID', 'ASD',
    'US DollGr', 'US DollVr', 'USN', 'HSD', 'US DKllar', 'US DollTr', 'UX Dollar',
    'US DoMlar', 'UY Dollar', 'US DOLYAR', 'US Jollar', 'US DolNar', 'USKDollar',
    'Isd', 'UH Dollar', 'US DollaT', 'ISD', 'US DAllar', 'usG', 'US DoOlar', 'USV',
    'usT', 'US DollYr', 'US DOLLOR', 'KSD', 'UK Dollar', 'US Aollar', 'US DollHr',
    'US DolBar', 'ZSD', 'USX', 'US DoUlar', 'MSD', 'MS Dollar', 'US Uollar',
    'US Xollar', 'USP', 'KS Dollar', 'USL', 'US DolFar', 'UOD', 'UZD', 'US DolVar',
    'Csd', 'USB', 'US DoJlar', 'PSD', 'US DoYlar', 'UAD', 'USU', 'YS Dollar',
    'IS Dollar', 'XSD', 'US DoAlar', 'Rs dollar', 'US DOLLTR', 'UPD', 'UR Dollar',
    'QS DOLLAR', 'UED', 'US DollaZ', 'US DollaV', 'USK', 'US DYllar', 'US DLllar',
    'USZ', 'US Follar', 'UQD', 'LS Dollar', 'AS Dollar', 'us dollRr', 'USH',
    'US DOLKAR', 'US DoPlar', 'USJ', 'UKD', 'US DollLr', 'US DollaY', 'XS Dollar',
    'VSD', 'US DolLar', 'CSD', 'US Eollar', 'US DoLlar', 'US DoNlar', 'us dollaR',
    'UE DOLLAR', 'DSD', 'UBD', 'US DollNr', 'SSD', 'VS Dollar', 'US DoDlar', 'UWD',
    'BSD', 'UND', 'US Gollar', 'USA', 'US DIllar', 'us dolSar', 'US DOLLBR', 'WSD',
    'US Mollar', 'UUD', 'uX dollar', 'Ls dollar', 'USIDollar', 'UST', 'USDDollar',
    'usC', 'UYD', 'UVD', 'USM', 'US DollMr', 'Zsd', 'TSD', 'Nsd', 'FS Dollar', 'USE',
    'ES Dollar', 'ZS Dollar', 'us dollNr', 'US DPllar', 'US DOllar', 'UR DOLLAR',
    'US Hollar', 'DS Dollar', 'US DWllar', 'usNdollar', 'UFD', 'UB Dollar',
    'US Collar', 'US Wollar', 'UG Dollar', 'UO Dollar', 'US DollaA', 'UN Dollar',
    'SS Dollar', 'US DCLLAR', 'uK dollar', 'Gsd', 'UJ Dollar', 'UW Dollar',
    'US DollDr', 'US DollJr', 'UT Dollar', 'US DolAar', 'Xsd', 'US DWLLAR', 'uVd',
    'us dollaG', 'US DollaF', 'UT DOLLAR', 'GS DOLLAR', 'UDD', 'US DolTar', 'ULD',
    'US DollaM', 'USNDollar', 'CS DOLLAR', 'BS Dollar', 'US DoSlar', 'US DoIlar',
    'US DolIar', 'US DTllar', 'usP', 'SS DOLLAR', 'USTDollar', 'US DolYar', 'UJD',
    'US DollFr', 'US DOLLUR', 'USY', 'Ysd', 'US DOALAR', 'US DEllar', 'Psd', 'FSD',
    'US DOSLAR', 'US DoVlar', 'US DALLAR', 'US DBllar', 'NS Dollar', 'US DOLPAR',
    'usEdollar', 'US DollAr', 'USO', 'USPDollar', 'UHD', 'US DOLLAT', 'US DollZr',
    'USG', 'UZ DOLLAR', 'us dollMr', 'USQ', 'US DollPr', 'USC', 'US Rollar',
    'US DollaQ', 'UU DOLLAR', 'USS', 'us dollKr', 'US DDllar', 'USSDOLLAR',
    'USLDollar', 'US Yollar', 'USI', 'JSD', 'US DolOar', 'UI Dollar', 'us Lollar',
    'uTd', 'US DolCar', 'usA', 'US DOULAR', 'US DODLAR', 'Qsd', 'uS dollar',
    'US DollaX', 'UV Dollar', 'uKd', 'UC DOLLAR', 'us dolDar', 'US DollaP',
    'USGDollar', 'UCD', 'US DollaO', 'US DOPLAR', 'UTD', 'US DolJar', 'US DOGLAR',
    'US DollRr', 'Cs dollar', 'US DoKlar', 'uV dollar', 'USCDollar', 'US DolWar',
    'US DolZar', 'UC Dollar', 'NSD', 'us Follar', 'US DollaD', 'UF Dollar',
    'USVDOLLAR', 'USXDollar', 'US DollIr', 'US DoRlar', 'USVDollar', 'OSD',
    'US DollaI', 'USZDollar', 'usAdollar', 'US DolMar', 'US DoQlar', 'Ksd', 'usV',
    'UA Dollar', 'US DolDar', 'uNd', 'US DolUar', 'US DJllar', 'us dXllar', 'GSD',
    'US DOLLAX', 'US DoFlar', 'UXD', 'usGdollar', 'usR', 'US Kollar', 'us dJllar',
    'Dsd', 'US Nollar', 'Msd', 'US DOMLAR', 'US DoXlar', 'US DollXr', 'URD',
    'US DolPar', 'uWd', 'USHDollar', 'US Pollar', 'UMD', 'US DollaL', 'US DollCr',
    'Lsd', 'us dNllar', 'OS Dollar', 'UO DOLLAR', 'US DULLAR', 'us dolMar',
    'QS Dollar', 'UP DOLLAR', 'US DolGar', 'USYDollar', 'US DoClar', 'US DQllar',
    'US DOLLHR', 'US DCllar', 'US DSLLAR', 'US DHllar', 'US DollaU', 'uQd',
    'HS Dollar', 'US DNllar', 'USW', 'US DFLLAR', 'US DoBlar', 'US DOLLKR',
    'US DollBr', 'USFDollar', 'CS Dollar', 'PS Dollar', 'US DoWlar', 'US Sollar',
    'US DollaR', 'US DollaB', 'USSDollar', 'USF', 'UGD', 'usB', 'us dFllar',
    'US DHLLAR', 'US DZllar', 'US DOLRAR', 'UL Dollar', 'US Zollar', 'US DollaE',
    'us dollaW', 'ESD', 'us dollaE', 'usQ', 'USIDOLLAR', 'US DRllar', 'us doOlar',
    'UY DOLLAR', 'US DolHar', 'UM Dollar', 'US Tollar', 'US DollEr', 'US SOLLAR',
    'UD Dollar', 'us dolZar', 'US DoTlar', 'US DoZlar', 'GS Dollar', 'USEDollar',
    'US DUllar', 'US DOLLAU', 'US DollaW', 'USQDOLLAR', 'US DTLLAR', 'uFd',
    'USODollar', 'USADollar', 'US DOLLAZ', 'RS Dollar', 'US DoHlar', 'US DOLLYR',
    'US DoGlar', 'USJDOLLAR', 'uLd', 'US DolKar', 'US DollaK', 'us Rollar',
    'Xs dollar', 'Bs dollar', 'US DolSar', 'US DollUr', 'uL dollar', 'uAd',
    'US DFllar', 'USWDollar', 'UP Dollar', 'us dollXr', 'USHDOLLAR', 'US DSllar',
    'USJDollar', 'usL', 'US DollaG', 'US DOTLAR', 'XS DOLLAR', 'NS DOLLAR',
    'TS DOLLAR', 'us Iollar', 'US DolQar', 'us dolHar', 'us Jollar', 'us dLllar',
    'US DollQr', 'US DGllar', 'uY dollar', 'US DMllar', 'AS DOLLAR', 'US DoElar',
    'USBDollar', 'us dolVar', 'uXd', 'USGDOLLAR', 'USQDollar', 'US DOLZAR',
    'US VOLLAR', 'us doLlar', 'us dolUar', 'Bsd', 'uC dollar', 'us doNlar', 'Rsd',
    'usF', 'uCd', 'usH', 'us doFlar', 'us Bollar', 'Usd', 'US UOLLAR', 'US DOLLRR',
    'us Wollar', 'Vsd', 'US DOLLAS', 'us dollLr', 'US Lollar', 'UQ Dollar',
    'Ms dollar', 'uZ dollar', 'US DollOr', 'usI', 'US DOLLWR', 'JS Dollar',
    'us dollaC', 'US DKLLAR', 'uJd', 'US Qollar', 'US DOVLAR', 'us doJlar',
    'uO dollar', 'Wsd', 'US DollWr', 'USLDOLLAR', 'uDd', 'USMDOLLAR', 'QSD',
    'UZ Dollar', 'US DolRar', 'Ts dollar', 'us dollUr', 'uMd', 'UU Dollar',
    'us dollaB', 'JS DOLLAR', 'USMDollar', 'us Nollar', 'us dYllar', 'US DollaH',
    'US DOOLAR', 'uId', 'US QOLLAR', 'US DollaC', 'US MOLLAR', 'us dolNar', 'Ssd',
    'US DollKr', 'USXDOLLAR', 'US DolXar', 'RS DOLLAR', 'US DPLLAR', 'UX DOLLAR',
    'uGd', 'US DOLCAR', 'usXdollar', 'UW DOLLAR', 'US DollaN', 'uHd', 'BS DOLLAR',
    'US DOWLAR', 'US DYLLAR', 'us dollaQ', 'USUDollar', 'us Qollar', 'USBDOLLAR',
    'us dollEr', 'US DOCLAR', 'USUDOLLAR', 'uB dollar', 'us dollaV', 'US DOILAR',
    'VS DOLLAR', 'us dolTar', 'US DOLBAR', 'uPd', 'us dollaT', 'usW', 'US DOQLAR',
    'US WOLLAR', 'usVdollar', 'USCDOLLAR', 'us dGllar', 'US Oollar', 'usZ', 'usU',
    'us dolKar', 'uT dollar', 'UF DOLLAR', 'UK DOLLAR', 'US DVLLAR', 'USFDOLLAR',
    'US DOLLPR', 'us doWlar', 'US DOLLJR', 'Ss dollar', 'us dolWar', 'Js dollar',
    'usIdollar', 'us Eollar', 'uOd', 'US DVllar', 'us dollaN', 'us Collar',
    'us doAlar', 'US Iollar', 'TS Dollar', 'US DOLLFR', 'us dollVr', 'US DOLLAV',
    'US DORLAR', 'us dolGar', 'uSd', 'US DOLEAR', 'usD', 'us dollaI', 'usBdollar',
    'US NOLLAR', 'us dolBar', 'US DOLOAR', 'US DOLMAR', 'US DOKLAR', 'USKDOLLAR',
    'uEd', 'Qs dollar', 'uRd', 'us dolPar', 'usE', 'US DollSr', 'WS DOLLAR',
    'UJ DOLLAR', 'usJ', 'US DOJLAR', 'US LOLLAR', 'USRDOLLAR', 'us doHlar',
    'USEDOLLAR', 'uU dollar', 'US DOLLAL', 'usDdollar', 'UH DOLLAR', 'us doUlar',
    'Ds dollar', 'us dolAar', 'IS DOLLAR', 'us dollZr', 'us doSlar', 'usY',
    'US DOLLAY', 'US DOLLAI', 'uUd', 'US DOLTAR', 'US DOLLAM', 'uN dollar',
    'us dDllar', 'us dollPr', 'us dollSr', 'Tsd', 'UM DOLLAR', 'US DOLLER',
    'US Bollar', 'US DOLLLR', 'US DRLLAR', 'US DOLLAB', 'USNDOLLAR', 'US DOLLAC',
    'UI DOLLAR', 'us dollaZ', 'us dSllar', 'us doClar', 'US DOELAR', 'uZd',
    'US DLLLAR', 'UA DOLLAR', 'us doGlar', 'us dolIar', 'Ks dollar', 'usYdollar',
    'USWDOLLAR', 'As dollar', 'Esd', 'US DJLLAR', 'uBd', 'Zs dollar', 'USYDOLLAR',
    'us dZllar', 'usN', 'usPdollar', 'us Dollar', 'Ns dollar', 'us dollWr', 'Asd',
    'US DOHLAR', 'US DOLLZR', 'US DDLLAR', 'usRdollar', 'us dollQr', 'usUdollar',
    'us doElar', 'US DMLLAR', 'Os dollar', 'us dollaH', 'uI dollar', 'US DONLAR',
    'us dCllar', 'FS DOLLAR', 'usSdollar', 'US GOLLAR', 'us dIllar', 'us dolJar',
    'US DOLLAF', 'US DOLLQR', 'Jsd', 'usOdollar', 'US DNLLAR', 'KS DOLLAR',
    'us dolLar', 'us dTllar', 'Ws dollar', 'usLdollar', 'us dUllar', 'us dMllar',
    'us dollaK', 'us Tollar', 'uE dollar', 'ES DOLLAR', 'us dQllar', 'YS DOLLAR',
    'US DOLLIR', 'Fsd', 'us dollCr', 'US OOLLAR', 'US DOLUAR', 'usS', 'USDDOLLAR',
    'usO', 'usX', 'US DOYLAR', 'us doRlar', 'US COLLAR', 'US KOLLAR', 'us dollAr',
    'US DOLSAR', 'US DOLLAW', 'us dollOr', 'us dollaA', 'US HOLLAR', 'Is dollar',
    'US DOLAAR', 'uG dollar', 'US DQLLAR', 'us dollJr', 'US DOLLDR', 'USTDOLLAR',
    'usM', 'US DOLGAR', 'USADOLLAR', 'Ys dollar', 'usFdollar', 'UL DOLLAR',
    'US EOLLAR', 'us dAllar', 'us dolXar', 'us doVlar', 'us Kollar', 'us Sollar',
    'uA dollar', 'US DZLLAR', 'us doXlar', 'US DOLLAN', 'us dollaM', 'us doTlar',
    'us dolCar', 'US DOLLXR', 'US DOLLSR', 'PS DOLLAR', 'us dPllar', 'uQ dollar',
    'uYd', 'US DOLVAR', 'US IOLLAR', 'US ZOLLAR', 'us dollDr', 'Es dollar',
    'us dollaO', 'us dollaF', 'us dollBr', 'us Zollar', 'US DOLQAR', 'Ps dollar',
    'US DOLLAQ', 'uW dollar', 'uP dollar', 'us dolOar', 'uJ dollar', 'US YOLLAR',
    'USODOLLAR', 'US DOLNAR', 'usTdollar'
]:
    currency_dicionario2[variation] = "USD"

In [72]:
base2_temp['Currency'] = base2_temp['Currency'].replace(currency_dicionario2)

base2_temp['Currency'] = base2_temp['Currency'].str.upper()

print(base2_temp['Currency'].head(10))

0    USD
1    USD
2    USD
3    USD
4    USD
5    USD
6    USD
7    USD
8    USD
9    USD
Name: Currency, dtype: object


-> Aqui vemos que a coluna foi toda padronizada para um só valor

In [73]:
valores_unicos = base2_temp['Currency'].unique()

print(valores_unicos)

['USD']


---
#### **Tratamento da coluna Marketcap**

-> A ideia inicial do tratamento, apos uma analise visual, era saber quais palavras, letras e simbolos estavam misturados nos inteiros para poder tratar.

->Então partimos para o tratamento mais rapido e pratico que era a retirada de virgulas e $

In [74]:
base2_temp['MarketCap'] = base2_temp['MarketCap'].astype(str).str.replace(',', '', regex=False)
base2_temp['MarketCap'] = base2_temp['MarketCap'].astype(str).str.replace('$', '', regex=False)

-> Verificação das primeiras linhas que ja tinham algumas virgulas e $

In [75]:
base2_temp.head(20)

,ID,Exchange,Currency,MarketCap,PE Ratio,EPS,Next Earnings,Previous Close,Day Range
0,1,NYSE,USD,370815220190,27.40,16.68,"May 19, 2025",567.57,X095.58-898.36
1,2,TSE,USD,1289775810656,90.13,"0,57",06-23-2025,$185.1Z,907.52-552.85
2,3,LSE,USD,97815908000,$15.67,18.17,07-21-2025,823.07,1380.78-1361.69
3,4,HKEX,USD,1658573090566,83.90,-4.82,23/07/2025,179.88,457.99-138.11
4,5,NASDAQ,USD,315341356874,79.86,5.67,08/06/2025,$1004.79,644.16-920.32
5,6,TSX,USD,1324763810463,76.46,18.45,03/08/2025,530.09,849.09-1331.72
6,7,NASDAQ,USD,1368238210098,"73,04",-Z.44,08-06-2025,445.71,1020.75-139.13
7,8,TSX,USD,1250223756881,65.29,19.56,05/07/2025,482.83,386.80-852.52
8,9,AMEX,USD,1543749651001,42.29,2.46,06/06/2025,361.15,868.17-589.25
9,10,AMEX,USD,1532453560561,55.93,-4.76,2025-05-17,1107.40,749.11-1066.47


-> Após removermos os símbolos de pontuação, como vírgulas e cifrões, partimos para o segundo problema da coluna Market Cap.

-> Identificamos que os valores estavam acompanhados de letras como M, K, B, entre outros sufixos que representam grandezas (mil, milhão, bilhão etc.). Após análise e discussão em grupo, concluímos que o valor médio de capitalização de mercado das empresas gira em torno de 13 dígitos.
Com base nessa constatação, desenvolvemos uma função chamada converter_market_cap (localizada na seção Funções generalizadas) que realiza os seguintes passos:

- Identifica e extrai o número e o sufixo (ex: "12.3B")

- Converte o número para o valor real multiplicando pelo fator correto (ex: B → 1 bilhão)

- Verifica se o valor convertido possui menos de 14 digitos

- Se sim, o valor é mantido convertido.

- Se não, o valor original é mantido, pois é considerado fora do padrão esperado.

Essa abordagem nos permitiu padronizar os valores válidos de Market.

###### **Verificação dos sufixos que estão na coluna**

In [76]:
resultado = extrair_sufixos_unicos(base2_temp['MarketCap'])
print(resultado)

['Billion', 'K', 'M', 'MK', 'Mb', 'b']


###### **Verificação e aplicação do tratamento**

In [77]:
base2_temp = converter_market_cap(base2_temp, 'MarketCap')

In [78]:
resultado = extrair_sufixos_unicos(base2_temp['MarketCap'])
print(resultado)

[]


-> Verificação final com uma amostra simples

In [79]:
base2_temp.head(20)

,ID,Exchange,Currency,MarketCap,PE Ratio,EPS,Next Earnings,Previous Close,Day Range
0,1,NYSE,USD,370815220190,27.40,16.68,"May 19, 2025",567.57,X095.58-898.36
1,2,TSE,USD,1289775810656,90.13,"0,57",06-23-2025,$185.1Z,907.52-552.85
2,3,LSE,USD,97815908000,$15.67,18.17,07-21-2025,823.07,1380.78-1361.69
3,4,HKEX,USD,1658573090566,83.90,-4.82,23/07/2025,179.88,457.99-138.11
4,5,NASDAQ,USD,315341356874,79.86,5.67,08/06/2025,$1004.79,644.16-920.32
5,6,TSX,USD,1324763810463,76.46,18.45,03/08/2025,530.09,849.09-1331.72
6,7,NASDAQ,USD,1368238210098,"73,04",-Z.44,08-06-2025,445.71,1020.75-139.13
7,8,TSX,USD,1250223756881,65.29,19.56,05/07/2025,482.83,386.80-852.52
8,9,AMEX,USD,1543749651001,42.29,2.46,06/06/2025,361.15,868.17-589.25
9,10,AMEX,USD,1532453560561,55.93,-4.76,2025-05-17,1107.40,749.11-1066.47


---
#### **Tratamento de PE Ratio**



-> Essa coluna apenas tem o problema de apresentar simbolos extras que não fazem parte da padronização, por exemplo $ e Z.

-> Nosso objetivo vai ser limpar os $ e tratar o Z baseado noq o monitor falar segunda(WIP)

-> O primeiro passo é encontrar valores que possam estar fora do padrao pa isso irei usar encontrar_simbolos para poder entender melhor o que tenho que fazer

In [80]:
encontrar_simbolos(base2_temp, 'PE Ratio')

{'$', ',', '.', 'Z'}

-> Após encontrar os simbolos, é visto que alguns numeros sao separados por virgula e precisamos mudar para ponto, bem como alguns numeros apresentam o simbolo $ e Z que precisam ser tratados

In [81]:
base2_temp['PE Ratio'] = base2_temp['PE Ratio'].astype(str).str.replace(',', '.', regex=False)
base2_temp['PE Ratio'] = base2_temp['PE Ratio'].astype(str).str.replace('$', '', regex=False)
base2_temp['PE Ratio'] = base2_temp['PE Ratio'].astype(str).str.replace('Z', '2', regex=False)


-> Verificando símbolos que podem ter sobrado

In [82]:
encontrar_simbolos(base2_temp, 'PE Ratio')

{'.'}

---
#### **Tratamento de EPS**


-> A ideia inicial de tratar essa coluna é analisar simbolos que não estão corretos para a padronização.

-> Para isso usaremos a mesma função de antes, encontrar_simbolos

In [83]:
encontrar_simbolos(base2_temp, 'EPS')

{'$', ',', '-', '.', 'Z'}

-> Foi visto simbolos como $, virgula e Z que não fazem parte da padronização da coluna, dessa forma precisaremos tratá-los

-> Seguiremos a mesma ideia de PE Ratio

In [84]:
base2_temp['EPS'] = base2_temp['EPS'].astype(str).str.replace(',', '.', regex=False)
base2_temp['EPS'] = base2_temp['EPS'].astype(str).str.replace('$', '', regex=False)
base2_temp['EPS'] = base2_temp['EPS'].astype(str).str.replace('Z', '2', regex=False)


-> Verificação dos símbolos que sobraram

In [85]:
encontrar_simbolos(base2_temp, 'EPS')

{'-', '.'}

---
#### **Tratamento de Previous Close**


-> Essa coluna também sofre do mesmo problema de PE Ratio e EPS, logo precisamos seguir a mesma logica que fizemos

In [86]:
encontrar_simbolos(base2_temp, 'Previous Close')

{'$', ',', '.', 'Z'}

-> Seguiremos a mesma logica, já que essa coluna apresenta o mesmo problema

In [87]:
base2_temp['Previous Close'] = base2_temp['Previous Close'].astype(str).str.replace(',', '.', regex=False)
base2_temp['Previous Close'] = base2_temp['Previous Close'].astype(str).str.replace('$', '', regex=False)
base2_temp['Previous Close'] = base2_temp['Previous Close'].astype(str).str.replace('Z', '2', regex=False)


In [88]:
encontrar_simbolos(base2_temp, 'Previous Close')

{'.'}

---
#### **Tratamento de Next Earnings**

-> Vamos tratar as datas na coluna para padronizar todas no formato dd-mm-yyyy

In [89]:
from dateutil import parser

def padronizar_data(data):
    try:
        data_convertida = parser.parse(str(data), dayfirst=False)
        return data_convertida.strftime('%d-%m-%Y')
    except Exception:
        return data

base2_temp['Next Earnings'] = base2_temp['Next Earnings'].apply(padronizar_data)

print(base2_temp['Next Earnings'].head(15))

0     19-05-2025
1     23-06-2025
2     21-07-2025
3     23-07-2025
4     06-08-2025
5     08-03-2025
6     06-08-2025
7     07-05-2025
8     06-06-2025
9     17-05-2025
10    02-06-2025
11    24-07-2025
12    18-06-2025
13    21-06-2025
14    25-07-2025
Name: Next Earnings, dtype: object


-> Nessa verificação é possível notar que ainda existem 2001 valores que ainda são considerados inválidos por diversos motivos

In [90]:
print(base2_temp['Next Earnings'].astype(str).str.extract(r'(\d{2}-\d{2}-\d{4})').isnull().sum())
print(base2_temp['Next Earnings'].unique())

0    2001
dtype: int64
['19-05-2025' '23-06-2025' '21-07-2025' ... '12/95/2013' '1/47/2009'
 '3/93/2006']


-> Executando outra função para garantir que todas as datas sejam passadas para o formato correto e transformando em None os valores inválidos para que possam ser substituídos pela moda

In [91]:
# Função para padronizar a data no formato dd-mm-yyyy
def padronizar_data2(data):
    try:
        data_convertida = parser.parse(data, dayfirst=False)  # tenta interpretar a data
        return data_convertida.strftime("%d-%m-%Y")  # retorna no formato desejado
    except Exception:
        return None  # retorna None se não conseguir converter

# Aplicar a função à coluna "Next Earnings"
base2_temp['Next Earnings'] = base2_temp['Next Earnings'].apply(padronizar_data2)

-> Verificando se o tratamento deu certo, checando o número de linhas que está em outro formato

In [92]:
# Função para verificar se a data está no formato dd-mm-yyyy
def check_date_format(data):
    if data is None:
        return True
    try:
        pd.to_datetime(data, format='%d-%m-%Y', exact=True)
        return True
    except ValueError:
        return False

# Aplicar a função para verificar o formato e contar as datas que não correspondem
invalid_format_count = base2_temp['Next Earnings'].apply(lambda x: not check_date_format(x)).sum()

print(f"\nNúmero de datas em 'Next Earnings' que NÃO estão no formato dd-mm-yyyy: {invalid_format_count}")

# Visualizar as datas com formato inválido
invalid_dates = base2_temp[base2_temp['Next Earnings'].apply(lambda x: not check_date_format(x))]['Next Earnings']
print("\nDatas com formato inválido:")
print(invalid_dates)


Número de datas em 'Next Earnings' que NÃO estão no formato dd-mm-yyyy: 0

Datas com formato inválido:
Series([], Name: Next Earnings, dtype: object)


-> Substituindo os valores nulos pela moda das datas e checagem final

In [93]:
# Preencher os valores nulos da coluna 'Next Earnings' com a moda
mode_next_earnings = base2_temp['Next Earnings'].mode()[0] # Calcula a moda
base2_temp['Next Earnings'].fillna(mode_next_earnings, inplace=True) # Preenche os nulos com a moda

# Verificar se ainda existem valores nulos na coluna 'Next Earnings'
print("\nContagem de valores nulos após o preenchimento com a moda:")
print(base2_temp['Next Earnings'].isnull().sum())

print("\nPrimeiras linhas do DataFrame após o preenchimento:")
print(base2_temp.head())


Contagem de valores nulos após o preenchimento com a moda:
0

Primeiras linhas do DataFrame após o preenchimento:
   ID Exchange Currency      MarketCap PE Ratio    EPS Next Earnings  \
0   1     NYSE      USD   370815220190    27.40  16.68    19-05-2025   
1   2      TSE      USD  1289775810656    90.13   0.57    23-06-2025   
2   3      LSE      USD    97815908000    15.67  18.17    21-07-2025   
3   4     HKEX      USD  1658573090566    83.90  -4.82    23-07-2025   
4   5   NASDAQ      USD   315341356874    79.86   5.67    08-06-2025   

  Previous Close        Day Range  
0         567.57   X095.58-898.36  
1         185.12    907.52-552.85  
2         823.07  1380.78-1361.69  
3         179.88    457.99-138.11  
4        1004.79    644.16-920.32  


<ipython-input-93-42a943686293>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  base2_temp['Next Earnings'].fillna(mode_next_earnings, inplace=True) # Preenche os nulos com a moda


---

-> Agora vamos converter para o formato da biblioteca datetime () para facilitar nossas futuras análises

In [94]:
# Limpa a coluna Date antes da conversão
base2_temp['Next Earnings'] = base2_temp['Next Earnings'].astype(str).str.strip()

# Converte para datetime com formato explícito
base2_temp['Next Earnings'] = pd.to_datetime(base2_temp['Next Earnings'], format='%d-%m-%Y', errors='coerce')

---
#### **Tratamento da coluna Day Range**

-> Pensando já no merge(essa foi a ultima coluna que faltava) nós pensamos, ok ela determina a diferença do menor para o maior valor, mas nos temos uma coluna na 1 base que tambem fala sobre isso, a Low e High.

-> Com esse pensamente, achamos justo refazer a coluna com os valores de low-high, colocando-a como float representado pela variância

-> Mas obviamente tiveram outros pontos que nos levaram a pensar assim. O principal foi o fato de ter numeros com letras e tais letras iam de a-z  deixando inviavel um dicionaio ou algo do tipo.



-> Substituindo pela variância, temos:

In [95]:
# Utilizando variáveis temporárias por boa prática
base_tmp111 = base1.copy()

# Substituindo pelo High e Low da base1
base2_temp['Day Range'] = base_tmp111['High']-base_tmp111['Low']

-> Visualização das mudanças

In [96]:
base2_temp.head()

,ID,Exchange,Currency,MarketCap,PE Ratio,EPS,Next Earnings,Previous Close,Day Range
0,1,NYSE,USD,370815220190,27.40,16.68,2025-05-19,567.57,1121.61
1,2,TSE,USD,1289775810656,90.13,0.57,2025-06-23,185.12,539.52
2,3,LSE,USD,97815908000,15.67,18.17,2025-07-21,823.07,1133.94
3,4,HKEX,USD,1658573090566,83.90,-4.82,2025-07-23,179.88,925.46
4,5,NASDAQ,USD,315341356874,79.86,5.67,2025-06-08,1004.79,1085.19


-> Passando as mudanças para a base original

In [97]:
base2 = base2_temp.copy()

---
#### **Visualização final da padronização e limpeza da Base 2**

In [98]:
base2.head()

,ID,Exchange,Currency,MarketCap,PE Ratio,EPS,Next Earnings,Previous Close,Day Range
0,1,NYSE,USD,370815220190,27.40,16.68,2025-05-19,567.57,1121.61
1,2,TSE,USD,1289775810656,90.13,0.57,2025-06-23,185.12,539.52
2,3,LSE,USD,97815908000,15.67,18.17,2025-07-21,823.07,1133.94
3,4,HKEX,USD,1658573090566,83.90,-4.82,2025-07-23,179.88,925.46
4,5,NASDAQ,USD,315341356874,79.86,5.67,2025-06-08,1004.79,1085.19


---
### **Etapa *Load***

-> Nessa última etapa, faremos a junção das duas bases limpas e padronizadas, gerando uma base final na qual trabalharemos a EDA



-> Para isso, nos basearemos no ID das duas bases para fazer o merge

In [99]:
# Junta as duas bases com base na coluna em comum ID
base_final = pd.merge(base1, base2, how = 'inner', on = 'ID')

-> Agora daremos uma visão geral na base final

In [100]:
base_final.head(10)

,ID,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Dividend,Split Ratio,Sector,Industry,Exchange,Currency,MarketCap,PE Ratio,EPS,Next Earnings,Previous Close,Day Range
0,1,2023-09-03,META,909.63,1380.05,258.44,595.86,61.54,8846040350,3.57,1:1,Communication,Media & Interactive Services,NYSE,USD,370815220190,27.40,16.68,2025-05-19,567.57,1121.61
1,2,2024-08-28,NVDA,678.96,725.03,185.51,295.83,843.78,7346935699,2.85,3:2,Technology,Semiconductors,TSE,USD,1289775810656,90.13,0.57,2025-06-23,185.12,539.52
2,3,2022-05-10,INTU,831.86,1348.94,215.00,874.05,336.19,8027589419,2.28,2:1,Technology,Software – Application,LSE,USD,97815908000,15.67,18.17,2025-07-21,823.07,1133.94
3,4,2017-12-26,NVDA,508.65,1111.85,186.39,511.49,952.63,1939500000,4.51,3:2,Technology,Semiconductors,HKEX,USD,1658573090566,83.90,-4.82,2025-07-23,179.88,925.46
4,5,2022-01-20,WORK,692.20,1425.94,340.75,378.58,1493.44,8984007687,1.93,3:2,Technology,Software – Application,NASDAQ,USD,315341356874,79.86,5.67,2025-06-08,1004.79,1085.19
5,6,2016-12-12,GOOG,262.36,1315.19,61.76,793.41,174.00,1434944091,4.09,3:2,Communication,Internet Services,TSX,USD,1324763810463,76.46,18.45,2025-08-03,530.09,1253.43
6,7,2015-11-13,TSLA,1112.92,1342.15,105.69,917.22,57.66,4621479471,0.33,2:1,Consumer Cyclical,Auto Manufacturers,NASDAQ,USD,1368238210098,73.04,-2.44,2025-06-08,445.71,1236.46
7,8,2015-01-31,NVDA,425.99,866.78,190.38,485.90,1434.97,5081802848,0.33,2:1,Technology,Semiconductors,TSX,USD,1250223756881,65.29,19.56,2025-07-05,482.83,676.40
8,9,2021-02-14,NFLX,676.41,1033.25,524.23,604.71,350.97,7748945628,2.64,2:1,Communication,Entertainment,AMEX,USD,1543749651001,42.29,2.46,2025-06-06,361.15,509.02
9,10,2024-11-17,ADBE,373.11,652.81,322.90,522.85,274.67,8357743507,2.41,2:1,Technology,Software – Application,AMEX,USD,1532453560561,55.93,-4.76,2025-05-17,1107.40,329.91


-> Confirmaremos se não há dados nulos:

In [101]:
base_final.isnull().sum()

,0
ID,0
Date,0
Ticker,0
Open,0
High,0
Low,0
Close,0
Adj Close,0
Volume,0
Dividend,0


-> Confirmaremos também se há dados duplicados

In [102]:
base_final.duplicated().sum()

np.int64(0)

-> Assim, visto que a base está padronizada e limpa, baixaremos e depois utilizarmos essa base no EDA:

In [103]:
base_final.to_csv('base_dados_final.csv', index=True)

In [104]:
from google.colab import files

# Faz o download do arquivo salvo
files.download('base_dados_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-> Baixaremos para xlsx para o Power BI

In [105]:
base_final.to_excel('base_dados_final.xlsx', index=True)